In [ ]:
# %pip install -U kaleido
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import copy
import significantdigits as sd

In [ ]:
Results_tableNoConf=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/IEEE/Results_tableNoConf_IEEE.pkl')
Results_tableWConf=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/IEEE/Results_tableWConf_IEEE.pkl')



In [ ]:
Result_Noconf_MCA=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/Result_tableNoConf_batch1.pkl')
Result_Wconf_MCA=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/Result_tableWConf_batch1.pkl')



In [ ]:
def collect_metric(df, mode):
    import pandas as pd
    import numpy as np

    if mode == 'IEEE':
        repetition = 2
        suffix = '(IEEE)'
    else:
        repetition = 11
        suffix = '(MCA)'

    columns = [
        f'degree_{suffix}', f'betweeness_{suffix}',
        f'clusteringcoef_{suffix}', f'eigenvec_{suffix}',
        f'smallworldness{suffix}', f'avg_shortestPathLength{suffix}'
        # 'subject', 'session', 'acquisition','reptition'
    ]

    metric = pd.DataFrame(columns=columns)
    metric = metric.astype({col: 'object' for col in columns if col not in ['subject', 'session', 'acquisition']})

    subjects = np.unique(df['subject'])
    i = 0

    for subj in subjects:
        filtered_df = df[df['subject'] == subj]

        for session in np.unique(filtered_df['session']):
            for acquisition in np.unique(filtered_df['acquisition']):
                if acquisition not in ['acq-RLsplit1', 'acq-LRsplit1']:

                    for rep in range(1, repetition):
                        filtered_rows = filtered_df[
                            (filtered_df['session'] == session) &
                            (filtered_df['acquisition'] == acquisition) &
                            (filtered_df['repetition'] == f'rep-{rep}')
                        ]
                        if filtered_rows.empty:
                            continue

                        degree_values = list(filtered_rows['degree_centralities'].values[0].values())
                        betweeness_values = list(filtered_rows['betweenness_centralities'].values[0].values())
                        eigen_values = list(filtered_rows['eigenvector_centralities'].values[0].values())
                        clustering_values = list(filtered_rows['clustering_coefficients'].values[0].values())
                        smallworld_values = filtered_rows['small_worldness'].values[0]
                        shortestpath_values = filtered_rows['avg_shortest_path_length'].values[0]


                        row_data = {
                            f'degree_{suffix}': degree_values,
                            f'betweeness_{suffix}': betweeness_values,
                            f'clusteringcoef_{suffix}': clustering_values,
                            f'eigenvec_{suffix}': eigen_values,
                            f'smallworldness{suffix}': smallworld_values,
                            f'avg_shortestPathLength{suffix}': shortestpath_values
                            # 'subject': subj,
                            # 'session': session,
                            # 'acquisition': acquisition,
                            # 'repetition':rep
                        }

                        metric.loc[i] = row_data
                        i += 1

    return metric


In [ ]:
Resulthc_Noconf_MCA=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/batchhc/Results_tableNoConf.pkl')
Resulthc_Wconf_MCA=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/batchhc/Results_tableWConf.pkl')

In [ ]:
metric_Wconf_ie=collect_metric(Results_tableWConf,mode='IEEE')
metric_Noconf_ie=collect_metric(Results_tableNoConf,mode='IEEE')
metric_Noconf_MCA=collect_metric(Result_Noconf_MCA,mode='MCA')
metric_Wconf_MCA=collect_metric(Result_Wconf_MCA,mode='MCA')
metrichc_Noconf_MCA=collect_metric(Resulthc_Noconf_MCA,mode='MCA')
metrichc_Wconf_MCA=collect_metric(Resulthc_Wconf_MCA,mode='MCA')

In [ ]:
# Suppose df is your original DataFrame
def stacked_metrics(df):
    # For the columns with lists (first 4 columns), flatten all lists
    # flattened = {
    #     col: sum(df[col].tolist(), [])  # flattens list of lists into one list
    #     for col in df.columns[:4]       # assuming first 4 columns are list-valued
    # }

    # # For the scalar columns, concatenate all values into a list
    # for col in df.columns[4:]:
    #     flattened[col] = df[col].tolist()

    # # Convert to DataFrame with one row
    # stacked_df = pd.DataFrame([flattened])
    # Suppose df is your original DataFrame

    # Define list-type metric columns and scalar metric columns
    list_columns = df.columns[:4]   # First 4: lists
    scalar_columns = df.columns[4:]  # Last 2: scalars

    # Stack list-columns into one list of lists
    stacked_data = {
        col: df[col].tolist() for col in list_columns
    }

    # Flatten scalar columns into one list
    for col in scalar_columns:
        stacked_data[col] = df[col].values.tolist()

    return stacked_data

stackmetric_Wconf_ie=stacked_metrics(metric_Wconf_ie)
stackmetric_Noconf_ie=stacked_metrics(metric_Noconf_ie)
stackmetric_Noconf_MCA=stacked_metrics(metric_Noconf_MCA)
stackmetric_Wconf_MCA=stacked_metrics(metric_Wconf_MCA)
stackmetrichc_Noconf_MCA=stacked_metrics(metrichc_Noconf_MCA)
stackmetrichc_Wconf_MCA=stacked_metrics(metrichc_Wconf_MCA)

In [ ]:
deg_arr_noie=   np.array(stackmetric_Noconf_ie['degree_(IEEE)'])
bet_arr_noie=   np.array(stackmetric_Noconf_ie['betweeness_(IEEE)'])
clu_arr_noie=   np.array(stackmetric_Noconf_ie['clusteringcoef_(IEEE)'])
eigen_arr_noie= np.array(stackmetric_Noconf_ie['eigenvec_(IEEE)'])
small_arr_noie= np.array(stackmetric_Noconf_ie['smallworldness(IEEE)'])
short_arr_noie= np.array(stackmetric_Noconf_ie['avg_shortestPathLength(IEEE)'])

deg_arr_cie=   np.array(stackmetric_Wconf_ie['degree_(IEEE)'])
bet_arr_cie=   np.array(stackmetric_Wconf_ie['betweeness_(IEEE)'])
clu_arr_cie=   np.array(stackmetric_Wconf_ie['clusteringcoef_(IEEE)'])
eigen_arr_cie= np.array(stackmetric_Wconf_ie['eigenvec_(IEEE)'])
Results_tableWConf
small_arr_cie= np.array(stackmetric_Wconf_ie['smallworldness(IEEE)'])
short_arr_cie= np.array(stackmetric_Wconf_ie['avg_shortestPathLength(IEEE)'])

deg_arr_cMCA=   np.array(stackmetric_Wconf_MCA['degree_(MCA)'])
bet_arr_cMCA=   np.array(stackmetric_Wconf_MCA['betweeness_(MCA)'])
clu_arr_cMCA=   np.array(stackmetric_Wconf_MCA['clusteringcoef_(MCA)'])
eigen_arr_cMCA= np.array(stackmetric_Wconf_MCA['eigenvec_(MCA)'])
small_arr_cMCA= np.array(stackmetric_Wconf_MCA['smallworldness(MCA)'])
short_arr_cMCA= np.array(stackmetric_Wconf_MCA['avg_shortestPathLength(MCA)'])

deg_arr_noMCA=   np.array(stackmetric_Noconf_MCA['degree_(MCA)'])
bet_arr_noMCA=   np.array(stackmetric_Noconf_MCA['betweeness_(MCA)'])
clu_arr_noMCA=   np.array(stackmetric_Noconf_MCA['clusteringcoef_(MCA)'])
eigen_arr_noMCA= np.array(stackmetric_Noconf_MCA['eigenvec_(MCA)'])
small_arr_noMCA= np.array(stackmetric_Noconf_MCA['smallworldness(MCA)'])
short_arr_noMCA= np.array(stackmetric_Noconf_MCA['avg_shortestPathLength(MCA)'])

deg_arrhc_cMCA=   np.array(stackmetrichc_Wconf_MCA['degree_(MCA)'])
bet_arrhc_cMCA=   np.array(stackmetrichc_Wconf_MCA['betweeness_(MCA)'])
clu_arrhc_cMCA=   np.array(stackmetrichc_Wconf_MCA['clusteringcoef_(MCA)'])
eigen_arrhc_cMCA= np.array(stackmetrichc_Wconf_MCA['eigenvec_(MCA)'])
small_arrhc_cMCA= np.array(stackmetrichc_Wconf_MCA['smallworldness(MCA)'])
short_arrhc_cMCA= np.array(stackmetrichc_Wconf_MCA['avg_shortestPathLength(MCA)'])

deg_arhcr_noMCA=   np.array(stackmetrichc_Noconf_MCA['degree_(MCA)'])
bet_arrhc_noMCA=   np.array(stackmetrichc_Noconf_MCA['betweeness_(MCA)'])
clu_arrhc_noMCA=   np.array(stackmetrichc_Noconf_MCA['clusteringcoef_(MCA)'])
eigen_arrhc_noMCA= np.array(stackmetrichc_Noconf_MCA['eigenvec_(MCA)'])
small_arrhc_noMCA= np.array(stackmetrichc_Noconf_MCA['smallworldness(MCA)'])
short_arrhc_noMCA= np.array(stackmetrichc_Noconf_MCA['avg_shortestPathLength(MCA)'])


In [ ]:
def collect_metricacross_MCA(df):
    """
    Calculate standard deviation of network metrics across repetitions for each subject, session, and acquisition.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe containing network metrics for different subjects, sessions, acquisitions and repetitions
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with standard deviations of metrics for each subject, session, and acquisition
    """
    # Define columns for metrics
    metric_columns = [
        'degree_(numericalVar)', 'betweeness_(numericalVar)', 
        'clusteringcoef_(numericalVar)', 'eigenvec_(numericalVar)', 
        'smallworldness(numericalVar)', 'avg_shortestPathLength(numericalVar)'
    ]
    
    # Define info columns
    info_columns = ['subject', 'session', 'acquisition']
    
    # Pre-filter to exclude unwanted acquisitions for efficiency
    df_filtered = df[~df['acquisition'].isin(['acq-RLsplit1', 'acq-LRsplit1'])]
    
    # Create empty list to collect results
    results = []
    
    # Group by subject, session, acquisition to avoid nested loops
    for (subj, session, acquisition), group in df_filtered.groupby(['subject', 'session', 'acquisition']):
        # Collections for each metric across repetitions
        all_metrics = {
            'degree': [], 'betweeness': [], 'eigen': [], 
            'clustering': [], 'smallworld': [], 'shortestpath': []
        }
        
        # Collect metrics for all repetitions
        for _, row in group.iterrows():
            rep_num = row.get('repetition')
            if not isinstance(rep_num, str) or not rep_num.startswith('rep-'):
                continue
                
            # Only process if we have the necessary data
            if (isinstance(row.get('degree_centralities'), dict) and 
                isinstance(row.get('betweenness_centralities'), dict) and
                isinstance(row.get('eigenvector_centralities'), dict) and
                isinstance(row.get('clustering_coefficients'), dict)):
                
                # Extract metrics
                all_metrics['degree'].append(list(row['degree_centralities'].values()))
                all_metrics['betweeness'].append(list(row['betweenness_centralities'].values()))
                all_metrics['eigen'].append(list(row['eigenvector_centralities'].values()))
                all_metrics['clustering'].append(list(row['clustering_coefficients'].values()))
                
                # Add scalar metrics if they exist
                if 'small_worldness' in row:
                    all_metrics['smallworld'].append(row['small_worldness'])
                if 'avg_shortest_path_length' in row:
                    all_metrics['shortestpath'].append(row['avg_shortest_path_length'])
        
        # Only calculate stats if we have data
        if not all_metrics['degree']:
            continue
            
        # Calculate standard deviations
        result_row = {
            'subject': subj,
            'session': session,
            'acquisition': acquisition,
            'degree_(numericalVar)': all_metrics['degree'],
            'betweeness_(numericalVar)': all_metrics['betweeness'],
            'eigenvec_(numericalVar)': all_metrics['eigen'],
            'clusteringcoef_(numericalVar)': all_metrics['clustering']
        }
        
        # Add scalar metrics if available
        if all_metrics['smallworld']:
            result_row['smallworldness(numericalVar)'] =all_metrics['smallworld']
        if all_metrics['shortestpath']:
            result_row['avg_shortestPathLength(numericalVar)'] = all_metrics['shortestpath']
            
        results.append(result_row)
    
    # Create DataFrame from results
    collectedmetric = pd.DataFrame(results)
    
    # Ensure proper data types for object columns
    for col in metric_columns:
        if col in collectedmetric.columns:
            collectedmetric[col] = collectedmetric[col].astype('object')
    
    return collectedmetric

In [ ]:
collect_Noconf=collect_metricacross_MCA(Results_tableNoConf)
collect_Wconf=collect_metricacross_MCA(Results_tableWConf)
m = collect_Noconf[
    (collect_Noconf['session'] == '1') &
    (collect_Noconf['subject'] == 'sub-102419') &
    (collect_Noconf['acquisition'] == 'acq-RL')]['degree_(numericalVar)'].values[0]#[0][0]
np.array(m).shape


In [ ]:
collect_mcaNoconf=collect_metricacross_MCA(Result_Noconf_MCA)
collect_MCAWconf=collect_metricacross_MCA(Result_Wconf_MCA)
collecthc_mcaNoconf=collect_metricacross_MCA(Resulthc_Noconf_MCA)
collecthc_MCAWconf=collect_metricacross_MCA(Resulthc_Wconf_MCA)
m = collect_mcaNoconf[
    (collect_mcaNoconf['session'] == '1') &
    (collect_mcaNoconf['subject'] == 'sub-102419') &
    (collect_mcaNoconf['acquisition'] == 'acq-RL')
]['degree_(numericalVar)'].values[0]
# print(m)
# # Unwrap the first (and only) element
# m = m[0]  # Now m should be shape (10, 100)

m = np.array(m)
print(m.shape) 
# len(m)
# collect_mcaNoconf[collect_mcaNoconf['session']=='1']
# collect_mcaNoconf

In [ ]:
def compute_test(mca,ieee):
    ieee=ieee[0]
    c1=sum(1 for val in mca if val < ieee)
    c2=sum(1 for val in mca if val > ieee)
    c_equal = sum(1 for val in mca if val == ieee)
    n_sample=10

    if (c1 == 0 and c2 == 10) or (c1 == 10 and c2 == 0):
        C = 0
    if c1==c2==0:
        C=10
    else:
        C = max(c1, c2, c_equal)
    
    p = (C + 1) / (n_sample +1)
    return C,p


In [ ]:
# def compute_test(mca,ieee):
#     ieee=ieee[0]
#     c1=sum(1 for val in mca if val < ieee)
#     c2=sum(1 for val in mca if val >= ieee)
#     if (all(val == ieee for val in mca)):
#         C=10
#     else:
#         C=max(c1,c2)
#     p=(C+1)/11
#     return C,p


In [ ]:
import pandas as pd
import numpy as np

# Create an empty list to store results
results_list = []

# Define the metrics we want to process
metrics = [
    'degree_(numericalVar)',
    'betweeness_(numericalVar)',
    'eigenvec_(numericalVar)',
    'clusteringcoef_(numericalVar)'
]

collect_mcaNoconf = collect_mcaNoconf[collect_mcaNoconf['session'] == '1']

for subj in collect_mcaNoconf['subject'].unique():
    subj_df = collect_mcaNoconf[collect_mcaNoconf['subject'] == subj]
    
    for acq in subj_df['acquisition'].unique():    
        acq_df = subj_df[subj_df['acquisition'] == acq]
        
        # Check if IEEE data is available for this subject and acquisition
        ieee_query = collect_Noconf[(collect_Noconf['session']=='1') & 
                                   (collect_Noconf['subject']==subj) & 
                                   (collect_Noconf['acquisition']==acq)]
        
        if len(ieee_query) == 0:
            print(f"No IEEE data available for subject {subj}, acquisition {acq}")
            continue
        
        # Process each metric
        for metric in metrics:
            if metric not in acq_df.columns:
                print(f"Metric {metric} not found in MCA data for subject {subj}, acquisition {acq}")
                continue
                
            if metric not in ieee_query.columns:
                print(f"Metric {metric} not found in IEEE data for subject {subj}, acquisition {acq}")
                continue
            
            # Get MCA values
            try:
                mca = np.array(acq_df[metric].values[0])
            except (IndexError, TypeError) as e:
                print(f"Error accessing MCA {metric} data for subject {subj}, acquisition {acq}: {e}")
                continue
                
            # Get IEEE values
            try:
                ieee = ieee_query[metric].values[0][0]
            except (IndexError, TypeError) as e:
                print(f"Error accessing IEEE {metric} data for subject {subj}, acquisition {acq}: {e}")
                continue
            
            # Process each region
            for region in range(100):
                try:
                    val_mca = mca[:, region]  # Extract values across 10 runs for one region
                except IndexError:
                    print(f"Region {region} not available in MCA {metric} data for subject {subj}, acquisition {acq}")
                    continue
                
                # Check if this region exists in IEEE data
                if region < len(ieee):
                    val_ieee = [ieee[region]]
                    
                    # Use the compute_test function
                    try:
                        c,p_value = compute_test(val_mca, val_ieee)
                    except Exception as e:
                        print(f"Error computing test for {metric}, region {region}, subject {subj}, acquisition {acq}: {e}")
                        continue
                    
                    # Store results in dictionary
                    result_row = {
                        'subject': subj,
                        'acquisition': acq,
                        'metric': metric,
                        'region': region,
                        'val_mca': val_mca.tolist(),  # Convert numpy array to list for DataFrame storage
                        'val_ieee': val_ieee,
                        'p_value': p_value,
                        'c_value':c
                    }
                    
                    # Append to results list
                    results_list.append(result_row)
                else:
                    print(f"Region {region} not available in IEEE {metric} data for subject {subj}, acquisition {acq}")
                    continue

# Create DataFrame from results list
results_df = pd.DataFrame(results_list)

# Optionally, save the DataFrame to CSV
# results_df.to_csv('metric_comparison_results.csv', index=False)

In [ ]:
# import pandas as pd
# import numpy as np
# from statsmodels.stats.multitest import multipletests

# # Assuming results_df is already created from previous code
# # First make sure the DataFrame is sorted appropriately
# results_df = results_df.sort_values(by=['subject', 'acquisition', 'metric'])

# # Set your significance threshold
# alpha_threshold = 0.01

# # Create lists to store the corrected results
# subjects_list = []
# acquisitions_list = []
# metrics_list = []
# significant_regions_list = []
# correction_methods = ['fdr_bh', 'bonferroni', 'holm']

# # Process each subject, acquisition, and metric combination separately
# for subj in results_df['subject'].unique():
#     for acq in results_df[results_df['subject'] == subj]['acquisition'].unique():
#         for metric in results_df[(results_df['subject'] == subj) & 
#                                (results_df['acquisition'] == acq)]['metric'].unique():
            
#             # Get p-values for this combination
#             subset = results_df[(results_df['subject'] == subj) & 
#                               (results_df['acquisition'] == acq) & 
#                               (results_df['metric'] == metric)]
            
#             if len(subset) == 0:
#                 continue
                
#             # Extract p-values
#             p_values = subset['p_value'].values
#             # Apply multiple comparison corrections
#             results_dict = {}
#             # for method in correction_methods:
#             # Apply correction
#             reject, pvals_corrected, _, _ = multipletests(p_values, alpha=alpha_threshold, method='bonferroni')
            
#             # Find significant regions after correction
#             significant_regions = subset.loc[reject, 'region'].tolist()
            
#             results_dict[f'bonferroni_significant'] = significant_regions
#             results_dict[f'bonferroni_count'] = len(significant_regions)
            
#             # Update the main DataFrame with corrected p-values
#             results_df.loc[subset.index, f'p_corrected_bonferroni'] = pvals_corrected
#             results_df.loc[subset.index, f'significant_bonferroni'] = reject
            
#             # Store summary results
#             subjects_list.append(subj)
#             acquisitions_list.append(acq)
#             metrics_list.append(metric)
#             significant_regions_list.append(results_dict)

# # Create a summary DataFrame
# summary_df = pd.DataFrame({
#     'subject': subjects_list,
#     'acquisition': acquisitions_list,
#     'metric': metrics_list,
#     'correction_results': significant_regions_list
# })

# # # Example: Display the number of significant regions using bonferroni correction
# # print("Summary of significant regions (bonferroni correction):")
# # for _, row in summary_df.iterrows():
# #     print(f"Subject: {row['subject']}, Acquisition: {row['acquisition']}, "
# #           f"Metric: {row['metric']}, Significant regions: {row['correction_results']['bonferroni_count']}")

# # Update the main results DataFrame with a significance flag (using FDR correction)
# results_df['is_significant'] = results_df['p_corrected_bonferroni'] < alpha_threshold

# # Optional: Save the resultimport numpy as np

# # results_df.to_csv('corrected_results.csv', index=False)
# # summary_df.to_csv('significance_summary.csv', index=False)

In [ ]:
results_df

In [ ]:
results_df['metric'].unique()

In [ ]:
res_df_Deg=results_df[(results_df['metric']=='degree_(numericalVar)') & (results_df['acquisition']=='acq-RL') &(results_df['region']==0)]
res_df_Deg

In [ ]:
df = res_df_Deg.apply(lambda row: row['val_mca'] + row['val_ieee'], axis=1)
matrix_df = pd.DataFrame(df.tolist())
matrix_df

In [ ]:
import numpy as np
rng = np.random.default_rng(seed=18)
from scipy.stats import friedmanchisquare
from statsmodels.stats.multitest import multipletests
result=[]
for region in range(100):
    res_df_Deg=results_df[(results_df['metric']=='degree_(numericalVar)') & (results_df['acquisition']=='acq-RL') &(results_df['region']==region)]
    df = res_df_Deg.apply(lambda row: row['val_mca'] + row['val_ieee'], axis=1)
    matrix_df = pd.DataFrame(df.tolist())
    res = friedmanchisquare(*[matrix_df[col] for col in matrix_df.columns])
    result.append(res.pvalue)
# Convert to array for easier processing
pvals = np.array(result)

# Apply FDR correction
rej, pvals_corrected, _, _ = multipletests(pvals, alpha=0.05, method='fdr_bh')
significant_regions = [i for i, p in enumerate(pvals_corrected) if p < 0.05]
print("Significant regions (p < 0.05):", significant_regions)




In [ ]:
res_df_Deg=results_df[(results_df['metric']=='degree_(numericalVar)') & (results_df['acquisition']=='acq-RL') &(results_df['region']==95)]
df = res_df_Deg.apply(lambda row: row['val_mca'] + row['val_ieee'], axis=1)
matrix = pd.DataFrame(df.tolist())

In [ ]:
import pandas as pd
import scikit_posthocs as sp

# df is your DataFrame with shape (100 subjects, 11 runs)
# Each column should represent a run

# Perform Nemenyi post-hoc test
nemenyi_result = sp.posthoc_nemenyi_friedman(matrix)

# Show the result
# import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(nemenyi_result, annot=True, cmap='viridis', fmt=".3f")
plt.title("Nemenyi Post-hoc Test: Pairwise Comparisons")
plt.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
# def plot_dist(df1,df2,df3,df4,metric):
#     # Labels for conditions and colors
#     data_list = [
#         (df1, "IEEE Confound", "blue"),
#         (df2, "IEEE No Confound", "green"),
#         (df3, "MCA Confound", "red"),
#         (df4, "MCA No Confound", "orange"),
#     ]

#     fig = go.Figure()

#     for arr, label, color in data_list:
#         for region in range(arr.shape[1]):
#             fig.add_trace(
#                 go.Box(
#                     y=arr[:, region],
#                     x=[f"Region {region+1}"] * arr.shape[0],
#                     name=label,
#                     boxpoints="outliers",
#                     marker_color=color,
#                     legendgroup=label,
#                     showlegend=(region == 0),  # Show legend only once per group
#                     line=dict(width=1),
#                     opacity=0.7
#                 )
#             )

#     # Layout adjustments
#     fig.update_layout(
#         title=f"Distribution of {metric} Centrality Across 100 Regions",
#         xaxis_title="Brain Region",
#         yaxis_title=f"{metric} Centrality",
#         boxmode="group",
#         showlegend=True,
#         width=1800,
#         height=600
#     )

#     fig.show()
import numpy as np
import plotly.graph_objects as go

# Example arrays (replace with real data)
# deg_arr_cie = ...
# deg_arr_noie = ...
# deg_arr_cMCA = ...
# deg_arr_noMCA = ...



In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_dist(df1, df2, df3, df4, metric):
    num_regions = df1.shape[1]
    regions_per_plot = 25
    num_subplots = num_regions // regions_per_plot


    fig = make_subplots(rows=num_subplots, cols=1, shared_yaxes=True,
                        # subplot_titles=[f"Regions {i*25+1}–{(i+1)*25}" for i in range(num_subplots)],
                        vertical_spacing=0.08)  # Increased spacing for labels

    for region in range(num_regions):
        subplot_row = region // regions_per_plot + 1
        region_in_subplot = region % regions_per_plot
        
        # Position adjustment for proper overlay
        pos_conf = region_in_subplot - 0.2  # Position for Confound group
        pos_no_conf = region_in_subplot + 0.2  # Position for No Confound group
        
        # CONFOUND GROUP - LEFT SIDE
        # -------------------------------
        # MCA Confound – box
        fig.add_trace(go.Box(
            y=df3[:, region],
            x=[pos_conf] * len(df3[:, region]),
            name="MCA Confound" if region == 0 else None,
            marker_color="red",
            boxpoints=False,
            legendgroup="MCA Confound",
            showlegend=(region == 0),
            line=dict(width=1),
            opacity=0.7,
            hoverinfo="y+name"
        ), row=subplot_row, col=1)

        # IEEE Confound – dots overlayed on MCA Confound box
        fig.add_trace(go.Scatter(
            y=df1[:, region],
            x=[pos_conf] * len(df1[:, region]),
            name="IEEE Confound" if region == 0 else None,
            mode="markers",
            marker=dict(
                size=3,
                color="blue",
                opacity=0.5
            ),
            legendgroup="IEEE Confound",
            showlegend=(region == 0),
            hoverinfo="y+name"
        ), row=subplot_row, col=1)
        
        # NO CONFOUND GROUP - RIGHT SIDE
        # -------------------------------
        # MCA No Confound – box
        fig.add_trace(go.Box(
            y=df4[:, region],
            x=[pos_no_conf] * len(df4[:, region]),
            name="MCA No Confound" if region == 0 else None,
            marker_color="orange",
            boxpoints=False,
            legendgroup="MCA No Confound",
            showlegend=(region == 0),
            line=dict(width=1),
            opacity=0.7,
            hoverinfo="y+name"
        ), row=subplot_row, col=1)

        # IEEE No Confound – dots overlayed on MCA No Confound box
        fig.add_trace(go.Scatter(
            y=df2[:, region],
            x=[pos_no_conf] * len(df2[:, region]),
            name="IEEE No Confound" if region == 0 else None,
            mode="markers",
            marker=dict(
                size=3,
                color="green",
                opacity=0.5
            ),
            legendgroup="IEEE No Confound",
            showlegend=(region == 0),
            hoverinfo="y+name"
        ), row=subplot_row, col=1)

    # Update layout and axes
    fig.update_layout(
        title=f"Distribution of {metric} Centrality Across {num_regions} Regions",
        height=200 * num_subplots,  # Increased height for label visibility
        width=2000,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        ),
        margin=dict(b=100)  # Add bottom margin for labels
    )
    
    # Set appropriate x-axis tick positions and labels for EVERY row
    for i in range(1, num_subplots + 1):
        # Calculate region range for this subplot
        start_region = regions_per_plot * (i-1)
        end_region = min(regions_per_plot * i, num_regions)
        
        # Update x-axis with tick values and labels
        fig.update_xaxes(
            row=i, 
            col=1,
            tickmode='array',
            tickvals=list(range(regions_per_plot)),  # Positions within the subplot
            ticktext=[f"R{j+1}" for j in range(start_region, end_region)],  # Actual region labels
            tickangle=45,  # Angled labels for better readability on ALL rows
            showticklabels=True,  # Show labels on ALL rows
            title_text="Brain Region" if i == num_subplots else ""  # Only show axis title on last row
        )
        
        # Set y-axis title only on the first row, but show tick labels on all
        fig.update_yaxes(
            title_text=f"{metric} Centrality" if i == 1 else "", 
            row=i, 
            col=1
        )

    # Add subtle grid lines for better readability
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(200,200,200,0.2)')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(200,200,200,0.2)')
    
    return fig

In [ ]:
plot_dist(deg_arr_cie,deg_arr_noie,deg_arr_cMCA,deg_arr_noMCA,'Degree')


In [ ]:
plot_dist(bet_arr_cie,bet_arr_noie,bet_arr_cMCA,bet_arr_noMCA,'Betweeness')


In [ ]:
plot_dist(clu_arr_cie,clu_arr_noie,clu_arr_cMCA,clu_arr_noMCA,'ClustringCoefficient')


In [ ]:
plot_dist(eigen_arr_cie,eigen_arr_noie,eigen_arr_cMCA,eigen_arr_noMCA,'EigenVector')

In [ ]:
plot_dist1(small_arr_cie,small_arr_noie,small_arr_cMCA,small_arr_noMCA,"SmallWorldness")


In [ ]:
plot_dist(short_arr_cie,short_arr_noie,short_arr_cMCA,short_arr_noMCA,'Average_shortestPath')

In [ ]:
import numpy as np
import plotly.graph_objects as go

def plot_dist1(df1, df2, df3, df4, metric):

    # Create figure
    fig = go.Figure()
    
    # Position adjustments for proper overlay
    pos_conf = 0  # Position for Confound group
    pos_no_conf = 1  # Position for No Confound group
    
    # CONFOUND GROUP - LEFT SIDE
    # -------------------------------
    # MCA Confound – box
    fig.add_trace(go.Box(
        y=df3,
        x=[pos_conf] * len(df3),
        name="MCA Confound",
        marker_color="red",
        boxpoints=False,
        line=dict(width=1),
        opacity=0.7,
        hoverinfo="y+name"
    ))

    # IEEE Confound – dots overlayed on MCA Confound box
    fig.add_trace(go.Scatter(
        y=df1,
        x=[pos_conf] * len(df1),
        name="IEEE Confound",
        mode="markers",
        marker=dict(
            size=5,
            color="blue",
            opacity=0.7
        ),
        hoverinfo="y+name"
    ))
    
    # NO CONFOUND GROUP - RIGHT SIDE
    # -------------------------------
    # MCA No Confound – box
    fig.add_trace(go.Box(
        y=df4,
        x=[pos_no_conf] * len(df4),
        name="MCA No Confound",
        marker_color="orange",
        boxpoints=False,
        line=dict(width=1),
        opacity=0.7,
        hoverinfo="y+name"
    ))

    # IEEE No Confound – dots overlayed on MCA No Confound box
    fig.add_trace(go.Scatter(
        y=df2,
        x=[pos_no_conf] * len(df2),
        name="IEEE No Confound",
        mode="markers",
        marker=dict(
            size=5,
            color="green",
            opacity=0.7
        ),
        hoverinfo="y+name"
    ))

    # Update layout
    fig.update_layout(
        title=f"Distribution of {metric}",
        height=600,
        width=800,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        ),
        xaxis=dict(
            tickmode='array',
            tickvals=[pos_conf, pos_no_conf],
            ticktext=["Confound", "No Confound"],
            title_text="Group"
        ),
        yaxis=dict(
            title_text=f"{metric} Value"
        )
    )
    
    # Add subtle grid lines
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(200,200,200,0.2)')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(200,200,200,0.2)')
    
    return fig


In [ ]:
# Compute ECDF for each sample
def ecdf(data):
    x = np.sort(data)  # Sort data
    y = np.arange(1, len(x) + 1) / len(x)  # Compute cumulative probability
    return x, y

In [ ]:
# import numpy as np
# import plotly.graph_objects as go
# def plot_metric_cdf(df1,df2,df3,df4,metric):
#     # Create a Plotly figure
#     fig = go.Figure()
#     if metric =='SmallWorldness' or metric == 'Average_shortestPath':
#         num=1
#         df1 = df1.reshape(-1, 1)  # shape becomes (70, 1)
#         df2 = df2.reshape(-1, 1)  # shape becomes (70, 1)
#         df3 = df3.reshape(-1, 1)  # shape becomes (70, 1)
#         df4 = df4.reshape(-1, 1)  # shape becomes (70, 1)

#     else:
#         num=100
#     # Add all ECDFs with transparency for each region
#     for idx in range(num):
#         xIE_wconf, yIE_wconf = ecdf(df1[:, idx])    # Green
#         xIE_noconf, yIE_noconf = ecdf(df2[:, idx]) # Red
#         xMCA_wconf, yMCA_wconf = ecdf(df3[:, idx]) # Blue
#         xMCA_noconf, yMCA_noconf = ecdf(df4[:, idx]) # Orange

#         # Green
#         fig.add_trace(go.Scatter(
#             x=xIE_wconf, y=yIE_wconf, mode='lines', showlegend=False,
#             line=dict(color='rgba(0, 128, 0, 0.02)'), hoverinfo='skip',dash='dash'
#         ))

#         # Red
#         fig.add_trace(go.Scatter(
#             x=xIE_noconf, y=yIE_noconf, mode='lines', showlegend=False,
#             line=dict(color='rgba(255, 0, 0, 0.02)'), hoverinfo='skip',dash='dash'
#         ))

#         # Blue
#         fig.add_trace(go.Scatter(
#             x=xMCA_wconf, y=yMCA_wconf, mode='lines', showlegend=False,
#             line=dict(color='rgba(0, 0, 255, 0.02)'), hoverinfo='skip'
#         ))

#         # Orange
#         fig.add_trace(go.Scatter(
#             x=xMCA_noconf, y=yMCA_noconf, mode='lines', showlegend=False,
#             line=dict(color='rgba(255, 165, 0, 0.02)'), hoverinfo='skip'
#         ))

#     # Add legend traces
#     fig.add_trace(go.Scatter(
#         x=[None], y=[None], mode='lines', name='With Confound IEEE',
#         line=dict(color='rgb(0, 128, 0)', width=2)
#     ))
#     fig.add_trace(go.Scatter(
#         x=[None], y=[None], mode='lines', name='No Confound IEEE',
#         line=dict(color='rgb(255, 0, 0)', width=2)
#     ))
#     fig.add_trace(go.Scatter(
#         x=[None], y=[None], mode='lines', name='With Confound MCA',
#         line=dict(color='rgb(0, 0, 255)', width=2)
#     ))
#     fig.add_trace(go.Scatter(
#         x=[None], y=[None], mode='lines', name='No Confound MCA',
#         line=dict(color='rgb(255, 165, 0)', width=2)
#     ))

#     # Update layout
#     fig.update_layout(
#         title=f"ECDF of {metric} Centrality values Across All 100 Regions",
#         xaxis_title=f"{metric} Centrality values",
#         yaxis_title="Cumulative Probability",
#         legend_title="Condition",
#         template="plotly_white"
#     )

#     # Show the figure
#     fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

def ecdf(data):
    """Compute x, y values for ECDF plot."""
    x = np.sort(data)
    y = np.arange(1, len(x)+1) / len(x)
    return x, y

def plot_metric_cdf(df1, df2, df3, df4, metric):
    fig = go.Figure()

    if metric == 'SmallWorldness' or metric == 'Average_shortestPath':
        num = 1
        df1 = df1.reshape(-1, 1)
        df2 = df2.reshape(-1, 1)
        df3 = df3.reshape(-1, 1)
        df4 = df4.reshape(-1, 1)
    else:
        num = 100

    # --- Plot faint ECDFs for each node ---
    for idx in range(num):
        xIE_wconf, yIE_wconf = ecdf(df1[:, idx])
        xIE_noconf, yIE_noconf = ecdf(df2[:, idx])
        xMCA_wconf, yMCA_wconf = ecdf(df3[:, idx])
        xMCA_noconf, yMCA_noconf = ecdf(df4[:, idx])

        fig.add_trace(go.Scatter(x=xIE_wconf, y=yIE_wconf, mode='lines', showlegend=False,
                                 line=dict(color='rgba(27, 158, 119, 0.1)', width=0.5), hoverinfo='skip'))

        fig.add_trace(go.Scatter(x=xIE_noconf, y=yIE_noconf, mode='lines', showlegend=False,
                                 line=dict(color='rgba(217, 95, 2, 0.1)', width=0.5), hoverinfo='skip'))

        fig.add_trace(go.Scatter(x=xMCA_wconf, y=yMCA_wconf, mode='lines', showlegend=False,
                                 line=dict(color='rgba(117, 112, 179, 0.1)', width=0.5), hoverinfo='skip'))

        fig.add_trace(go.Scatter(x=xMCA_noconf, y=yMCA_noconf, mode='lines', showlegend=False,
                                 line=dict(color='rgba(231, 41, 138, 0.1)', width=0.5), hoverinfo='skip'))

    # --- Plot mean ECDF for each group ---
    def add_mean_ecdf(data, color, name):
        x_sorted = np.sort(data, axis=0)         # shape: (70, num)
        x_mean = np.mean(x_sorted, axis=1)
        y_vals = np.linspace(0, 1, len(x_mean))
        fig.add_trace(go.Scatter(x=x_mean, y=y_vals, mode='lines', name=name,
                                 line=dict(color=color, width=2.5)))

    add_mean_ecdf(df1, 'rgb(27, 158, 119)', 'With Confound IEEE')
    add_mean_ecdf(df2, 'rgb(217, 95, 2)',   'No Confound IEEE')
    add_mean_ecdf(df3, 'rgb(117, 112, 179)', 'With Confound MCA')
    add_mean_ecdf(df4, 'rgb(231, 41, 138)',  'No Confound MCA')

    # --- Layout ---
    fig.update_layout(
        title=f"ECDF of {metric} Centrality values Across All {num if num > 1 else 'Subjects'}",
        xaxis_title=f"{metric} Values",
        yaxis_title="Cumulative Probability",
        legend_title="Condition",
        template="plotly_white"
    )

    fig.show()


In [ ]:
plot_metric_cdf(deg_arr_cie,deg_arr_noie,deg_arr_cMCA,deg_arr_noMCA,'Degree')


In [ ]:
plot_metric_cdf(deg_arr_cie,deg_arr_noie,deg_arr_cMCA,deg_arr_noMCA,'Degree')
plot_metric_cdf(bet_arr_cie,bet_arr_noie,bet_arr_cMCA,bet_arr_noMCA,'Betweeness')
plot_metric_cdf(clu_arr_cie,clu_arr_noie,clu_arr_cMCA,clu_arr_noMCA,'ClustringCoefficient')
plot_metric_cdf(eigen_arr_cie,eigen_arr_noie,eigen_arr_cMCA,eigen_arr_noMCA,'EigenVector')
plot_metric_cdf(small_arr_cie,small_arr_noie,small_arr_cMCA,small_arr_noMCA,"SmallWorldness")
plot_metric_cdf(short_arr_cie,short_arr_noie,short_arr_cMCA,short_arr_noMCA,'Average_shortestPath')

In [ ]:
# Choose a subset to plot (e.g., first 10 ECDFs)
num_to_plot = 10 # Adjust as needed
selected_indices = [3724,2584,3550]

# Get a list of colors from Plotly's qualitative palette
colors = px.colors.qualitative.Plotly

# Create a Plotly figure
fig = go.Figure()

# Add ECDF traces for selected elements
for i, idx in enumerate(selected_indices):
    color = colors[i % len(colors)]  # Cycle through colors
    x_wconf, y_wconf = ecdf(subj_valwconfsig_matrix[idx])
    x_noconf, y_noconf = ecdf(subj_valNoconfsig_matrix[idx])
    
    fig.add_trace(go.Scatter(x=x_wconf, y=y_wconf, mode='lines', name=f'With Confound {idx}', line=dict(color=color)))
    fig.add_trace(go.Scatter(x=x_noconf, y=y_noconf, mode='lines', name=f'No Confound {idx}', line=dict(color=color, dash='dash')))

# Update layout for better readability
fig.update_layout(
    title="ECDF of Significant digits  Values across all 4949 Regions pairs",
    xaxis_title=" Significant digits Value",
    yaxis_title="Cumulative Probability",
    legend_title="Legend",
    template="plotly_white"
)

# Show the figure
fig.show()

In [ ]:
import numpy as np
import pandas as pd

# Assuming 'df' has already been extracted
df = df_Noconf_num[(df_Noconf_num['acquisition'] == 'acq-RL') & 
                             (df_Noconf_num['session'] == '1')]

df = df.drop(columns=['session', 'acquisition', 'subject' # Remove the unnecessary column
])



# Initialize a new DataFrame for the final stacked result
stacked_row = {}

# Loop through each column
for col in df.columns:
    # If column contains matrices or arrays, stack them vertically
    stacked_row[col] = np.vstack(df[col].values)

# Convert the dictionary to a DataFrame with one row
stacked_df = pd.DataFrame([stacked_row])
# Create a DataFrame to store the averaged values
avrg_Noconf_num = pd.DataFrame({
    col: [np.mean(stacked_df.iloc[0][col], axis=0)] for col in ['degree_(numericalVar)',	'betweeness_(numericalVar)',	'clusteringcoef_(numericalVar)' , 'eigenvec_(numericalVar)']
})
# Add 'small_worldness' and 'avg_shortest_path_length' to avrg_Wconf_anat
avrg_Noconf_num['smallworldness(numericalVar)'] =stacked_df['smallworldness(numericalVar)']
avrg_Noconf_num['avg_shortestPathLength(numericalVar)'] = stacked_df['avg_shortestPathLength(numericalVar)']

##########################################################################################3
# Assuming 'df' has already been extracted
df1= df_Wconf_num[(df_Wconf_num['acquisition'] == 'acq-RL') & 
                             (df_Wconf_num['session'] == '1')]

df1 = df1.drop(columns=['session', 'acquisition', 'subject'])



# Initialize a new DataFrame for the final stacked result
stacked_row = {}

# Loop through each column
for col in df1.columns:
    # If column contains matrices or arrays, stack them vertically
    stacked_row[col] = np.vstack(df1[col].values)

# Convert the dictionary to a DataFrame with one row
stacked_df1 = pd.DataFrame([stacked_row])
# Create a DataFrame to store the averaged values
avrg_Wconf_num = pd.DataFrame({
    col: [np.mean(stacked_df1.iloc[0][col], axis=0)] for col in ['degree_(numericalVar)',	'betweeness_(numericalVar)',	'clusteringcoef_(numericalVar)' , 'eigenvec_(numericalVar)']
})
# Add 'small_worldness' and 'avg_shortest_path_length' to avrg_Wconf_anat
avrg_Wconf_num['smallworldness(numericalVar)'] =stacked_df1['smallworldness(numericalVar)']
avrg_Wconf_num['avg_shortestPathLength(numericalVar)'] = stacked_df1['avg_shortestPathLength(numericalVar)']


In [ ]:
import numpy as np
import pandas as pd

# Assuming 'df' has already been extracted
df = df_Noconf_anat[(df_Noconf_anat['acquisition'] == 'acq-RL') & 
                             (df_Noconf_anat['session'] == '1')&(df_Noconf_anat['iteration']=="iter_1")]

df = df.drop(columns=['session', 'acquisition', 'iteration'])



# Initialize a new DataFrame for the final stacked result
stacked_row1= {}

# Loop through each column
for col in df.columns:
    # If column contains matrices or arrays, stack them vertically
    stacked_row1[col] = np.vstack(df[col].values)

# Convert the dictionary to a DataFrame with one row
stacked_df = pd.DataFrame([stacked_row1])
# Create a DataFrame to store the averaged values
avrg_Noconf_anat = pd.DataFrame({
    col: [np.mean(stacked_df.iloc[0][col], axis=0)] for col in ['degree_(AnatomicalVar)',	'betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)' , 'eigenvec_(AnatomicalVar)']
})
# Add 'small_worldness' and 'avg_shortest_path_length' to avrg_Wconf_anat
avrg_Noconf_anat['smallworldness(AnatomicalVar)'] =stacked_df['smallworldness(AnatomicalVar)']
avrg_Noconf_anat['avg_shortestPathLength(AnatomicalVar)'] = stacked_df['avg_shortestPathLength(AnatomicalVar)']
###########################################################################################3
# Assuming 'df' has already been extracted
df1 = df_Wconf_anat[(df_Wconf_anat['acquisition'] == 'acq-RL') & 
                             (df_Wconf_anat['session'] == '1')& (df_Wconf_anat['iteration']=="iter_1")]

df1 = df1.drop(columns=['session', 'acquisition', 'iteration'])



# Initialize a new DataFrame for the final stacked result
stacked_row1= {}

# Loop through each column
for col in df1.columns:
    # If column contains matrices or arrays, stack them vertically
    stacked_row1[col] = np.vstack(df1[col].values)

# Convert the dictionary to a DataFrame with one row
stacked_df1 = pd.DataFrame([stacked_row1])
# Create a DataFrame to store the averaged values(df_Noconf_anat['iteration']=="iter_1")
# Create a DataFrame to store the averaged values
avrg_Wconf_anat = pd.DataFrame({
    col: [np.mean(stacked_df1.iloc[0][col], axis=0)] for col in ['degree_(AnatomicalVar)',	'betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)' , 'eigenvec_(AnatomicalVar)']
})
# Add 'small_worldness' and 'avg_shortest_path_length' to avrg_Wconf_anat
avrg_Wconf_anat['smallworldness(AnatomicalVar)'] =stacked_df1['smallworldness(AnatomicalVar)']
avrg_Wconf_anat['avg_shortestPathLength(AnatomicalVar)'] = stacked_df1['avg_shortestPathLength(AnatomicalVar)']

avrg_Wconf_num = avrg_Wconf_num.rename(columns={'degree_(numericalVar)': 'degree_(numericalVarW)', 'betweeness_(numericalVar)': 'betweeness_(numericalVarW)' , 'clusteringcoef_(numericalVar)':'clusteringcoef_(numericalVarW)'   , 'eigenvec_(numericalVar)':'eigenvec_(numericalVarW)'  ,'smallworldness(numericalVar)':'smallworldness(numericalVarW)','avg_shortestPathLength(numericalVar)':'avg_shortestPathLength(numericalVarW)'

	})
avrg_Wconf_anat = avrg_Wconf_anat.rename(columns={'degree_(AnatomicalVar)': 'degree_(AnatomicalVarW)', 'betweeness_(AnatomicalVar)': 'betweeness_(AnatomicalVarW)' , 'clusteringcoef_(AnatomicalVar)':'clusteringcoef_(AnatomicalVarW)'   , 'eigenvec_(AnatomicalVar)':'eigenvec_(AnatomicalVarW)','smallworldness(AnatomicalVar)':'smallworldness(AnatomicalVarW)','avg_shortestPathLength(AnatomicalVar)':'avg_shortestPathLength(AnatomicalVarW)'  })

# stacked_df1
# np.array(stacked_df1.iloc[0]['smallworldness(AnatomicalVar)']).shape


In [ ]:
def change(arr,col):
    # Create the 'region' column with labels
    
    regions = [f"region_{i+1}" for i in range(len(arr))]

    # Create the DataFrame
    df = pd.DataFrame({'region': regions, f'{col}': arr})
    return df

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots

# List of metrics
metrics = ['degree', 'betweeness', 'eigenvec', 'clusteringcoef']

# Create subplots
fig = make_subplots(
    rows=len(metrics), 
    cols=1,  
    vertical_spacing=0.05   
    # subplot_titles=[
    #     f"Degree Centrality with anatomical mean = {avr_BetweenSubject['degree_(AnatomicalVar)'].mean()}",
    #     f"Betweeness Centrality with anatomical mean = {avr_BetweenSubject['betweeness_(AnatomicalVar)'].mean()}",
    #     f"Eigen Vector Centrality with anatomical mean = {avr_BetweenSubject['eigenvec_(AnatomicalVar)'].mean()}",
    #     f"Clustering Coefficient with anatomical mean = {avr_BetweenSubject['clusteringcoef_(AnatomicalVar)'].mean()}"
    # ]
)



for i, metric in enumerate(metrics):
    # Extract columns for each metric
    columns_to_extract_bet1 =  [col for col in avrg_Wconf_anat.columns if metric in col]
    columns_to_extract_with1 = [col for col in avrg_Wconf_num.columns if metric in col]
    statics_bet1 = change(avrg_Wconf_anat[columns_to_extract_bet1].iloc[0][avrg_Wconf_anat[columns_to_extract_bet1].columns[0]],avrg_Wconf_anat[columns_to_extract_bet1].columns[0])
    statics_with1 = change(avrg_Wconf_num[columns_to_extract_with1].iloc[0][avrg_Wconf_num[columns_to_extract_with1].columns[0]],avrg_Wconf_num[columns_to_extract_with1].columns[0])

    columns_to_extract_bet = [col for col in avrg_Noconf_anat.columns if metric in col]
    columns_to_extract_with =  [col for col in avrg_Noconf_num.columns if metric in col]
    statics_bet = change(avrg_Noconf_anat[columns_to_extract_bet].iloc[0][avrg_Noconf_anat[columns_to_extract_bet].columns[0]],avrg_Noconf_anat[columns_to_extract_bet].columns[0])
    statics_with = change(avrg_Noconf_num[columns_to_extract_with].iloc[0][avrg_Noconf_num[columns_to_extract_with].columns[0]],avrg_Noconf_num[columns_to_extract_with].columns[0])
    
    # Melt the DataFrames
    bet_melted1 = statics_bet1.melt(id_vars=['region'], var_name='metric', value_name='value')
    with_melted1 = statics_with1.melt(id_vars=['region'], var_name='metric', value_name='value')

    bet_melted = statics_bet.melt(id_vars=['region'], var_name='metric', value_name='value')
    with_melted = statics_with.melt(id_vars=['region'], var_name='metric', value_name='value')   
    # Plot box plots for Between Subject
    fig.add_trace(
        go.Box(
            x=bet_melted1['metric'], 
            y=bet_melted1['value'],
            showlegend=False,
            name=f"Between Subject {metric.capitalize()}",
            boxpoints='all',  # Add stripplot-like points
            jitter=0.3,
            pointpos=0,
            marker=dict(
                        color='rgb(34,139,34)'),
            width=0.2
        ),
        row=i + 1, col=1
    )
        # Plot box plots for Between Subject
    fig.add_trace(
        go.Box(
            x=bet_melted['metric'], 
            y=bet_melted['value'],
            showlegend=False,
            name=f"Between Subject {metric.capitalize()}",
            boxpoints='all',  # Add stripplot-like points
            jitter=0.3,
            pointpos=0,
            marker=dict(
                        color='rgb(34,139,34)'),
            width=0.2
        ),
        row=i + 1, col=1
    )
    # Plot box plots for Within Subject
    fig.add_trace(
        go.Box(
            x=with_melted1['metric'], 
            y=with_melted1['value'],
            showlegend=False,
            width=0.2,  # Smaller box width,
            name=f"Within Subject {metric.capitalize()}",
            boxpoints='suspectedoutliers', # only suspected outliers
            jitter=0.3,
            pointpos=0.1,
            marker=dict(
                        color='rgb(8,81,156)',
                        outliercolor='rgb(8,81,156)',
                        line=dict(
                            outliercolor='rgb(8,81,156)',
                            outlierwidth=2)),
            
            text=with_melted1['region'],  # Hover text showing subject ID
            hovertemplate=(
                "<b>region:</b> %{text}<br>"  # Display subject ID
                "<b>Metric:</b> %{x}<br>"      # Display metric
                "<b>Value:</b> %{y}<br>"       # Display value
                "<extra></extra>"              # Remove default extra info
        )
        ),
        row=i + 1, col=1
    )
        # Plot box plots for Within Subject
    fig.add_trace(
        go.Box(
            x=with_melted['metric'], 
            y=with_melted['value'],
            showlegend=False,
            width=0.2,  # Smaller box width,
            name=f"Within Subject {metric.capitalize()}",
            boxpoints='suspectedoutliers', # only suspected outliers
            jitter=0.3,
            pointpos=0.1,
            marker=dict(
                        color='rgb(8,81,156)',
                        outliercolor='rgb(8,81,156)',
                        line=dict(
                            outliercolor='rgb(8,81,156)',
                            outlierwidth=2)),
            
            text=with_melted['region'],  # Hover text showing subject ID
            hovertemplate=(
                "<b>region:</b> %{text}<br>"  # Display subject ID
                "<b>Metric:</b> %{x}<br>"      # Display metric
                "<b>Value:</b> %{y}<br>"       # Display value
                "<extra></extra>"              # Remove default extra info
        )
        ),
        row=i + 1, col=1
    )

# Update layout
fig.update_layout(
    title="Anatomical and Numerical Variability (Local Graph Metric)",
    height=300 * len(metrics),
    showlegend=True,
    legend_title="Legend",
    margin=dict(l=50, r=50, t=50, b=100),
    template='plotly'
)

# Iterate through rows to remove x-axis tick labels for all subplots
for row in range(1, len(metrics) + 1):
    fig.update_xaxes(showticklabels=False, row=row, col=1)

fig.update_xaxes(
    title_text="Anatomical Variability with confounds" + "&nbsp;" *65 +  "Anatomical Variability No confounds" + "&nbsp;" *65  + "Numerical Variability with confounds" + "&nbsp;" *65 + "Numerical Variability No confounds",
    row=len(metrics),
    col=1,
    side='bottom'
)
# Add section titles for anatomical and numerical data
fig.add_annotation(
    text="Degree Centrality",
    xref="paper", yref="paper",
    x=0.0, y=1.018,  # Centered horizontally above the first two rows
    showarrow=False,
)

fig.add_annotation(
    text="Betweeness Centrality",
    xref="paper", yref="paper",
    x=0.0, y=0.77 , # Centered horizontally above the last two rows
    showarrow=False,
)
fig.add_annotation(
    text="Eigen vector Centrality",
    xref="paper", yref="paper",
    x=0.0, y=0.5,  # Centered horizontally above the last two rows
    showarrow=False,
)

fig.add_annotation(
    text="Clustering coefficient",
    xref="paper", yref="paper",
    x=0.0, y=0.23,  # Centered horizontally above the last two rows
    showarrow=False,
)


# # Remove annotations for avr_bet
# import plotly.io as pio
# pio.write_html(fig, "LocalMetricplot.html")
fig.show()

In [ ]:
stdofavr_WithinSubject=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/NumGmetricstd_NoConf_batchhc.pkl')
avr_WithinSubject=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/NumGmetricavr_NoConf_batchhc.pkl')
stdofavr_BetweenSubject=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/AnatGmetricstd_NoConf_batchhc.pkl')
avr_BetweenSubject=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/AnatGmetricavr_NoConf_batchhc.pkl')

In [ ]:
stdofavr_WithinSubject1 = stdofavr_WithinSubject[
    (stdofavr_WithinSubject['session'] == '1') & 
    (stdofavr_WithinSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness(numericalVar)','avg_shortestPathLength(numericalVar)'])

# Rename specific columns
stdofavr_WithinSubject1 = stdofavr_WithinSubject1.rename(columns={'degree_(numericalVar)': 'degree_(numericalVar1)', 'betweeness_(numericalVar)': 'betweeness_(numericalVar1)' , 'clusteringcoef_(numericalVar)':'clusteringcoef_(numericalVar1)'   , 'eigenvec_(numericalVar)':'eigenvec_(numericalVar1)'  })

stdofavr_BetweenSubject1 = stdofavr_BetweenSubject[
    (stdofavr_BetweenSubject['session'] == '1') & 
    (stdofavr_BetweenSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness_(AnatomicalVar)','avg_shortestPathLength_(AnatomicalVar)'])

stdofavr_BetweenSubject1 = stdofavr_BetweenSubject1.rename(columns={'degree_(AnatomicalVar)': 'degree_(AnatomicalVar1)', 'betweeness_(AnatomicalVar)': 'betweeness_(AnatomicalVar1)' , 'clusteringcoef_(AnatomicalVar)':'clusteringcoef_(AnatomicalVar1)'   , 'eigenvec_(AnatomicalVar)':'eigenvec_(AnatomicalVar1)'  })


In [ ]:
df_stdofavr_WithinSubject=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/NumGmetricstd_WConf_batchhc.pkl')
df_stdofavr_BetweenSubject=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/AnatGmetricstd_WConf_batchhc.pkl')

In [ ]:
stdofavr_WithinSubjectWconf = df_stdofavr_WithinSubject[
    (df_stdofavr_WithinSubject['session'] == '1') & 
    (df_stdofavr_WithinSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness(numericalVar)','avg_shortestPathLength(numericalVar)'])
stdofavr_BetweenSubjectWconf = df_stdofavr_BetweenSubject[
    (df_stdofavr_BetweenSubject['session'] == '1') & 
    (df_stdofavr_BetweenSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness_(AnatomicalVar)','avg_shortestPathLength_(AnatomicalVar)'])

stdofavr_WithinSubjectWconf = stdofavr_WithinSubjectWconf.rename(columns={'degree_(numericalVar)': 'degree_(numericalVarW)', 'betweeness_(numericalVar)': 'betweeness_(numericalVarW)' , 'clusteringcoef_(numericalVar)':'clusteringcoef_(numericalVarW)'   , 'eigenvec_(numericalVar)':'eigenvec_(numericalVarW)'  })
stdofavr_BetweenSubjectWconf = stdofavr_BetweenSubjectWconf.rename(columns={'degree_(AnatomicalVar)': 'degree_(AnatomicalVarW)', 'betweeness_(AnatomicalVar)': 'betweeness_(AnatomicalVarW)' , 'clusteringcoef_(AnatomicalVar)':'clusteringcoef_(AnatomicalVarW)'   , 'eigenvec_(AnatomicalVar)':'eigenvec_(AnatomicalVarW)'  })


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

# List of metrics
metrics = ['degree', 'betweeness', 'eigenvec', 'clusteringcoef']
stdofavr_BetweenSubjectWconf
# Create subplots
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=len(metrics), 
    cols=1,  
    vertical_spacing=0.05,     
    subplot_titles=[
        f"Degree Centrality with anatomical mean = {avr_BetweenSubject['degree_(AnatomicalVar)'].mean()}",
        f"Betweeness Centrality with anatomical mean = {avr_BetweenSubject['betweeness_(AnatomicalVar)'].mean()}",
        f"Eigen Vector Centrality with anatomical mean = {avr_BetweenSubject['eigenvec_(AnatomicalVar)'].mean()}",
        f"Clustering Coefficient with anatomical mean = {avr_BetweenSubject['clusteringcoef_(AnatomicalVar)'].mean()}"
    ]
)



for i, metric in enumerate(metrics):
    # Extract columns for each metric
    columns_to_extract_bet1 = ['iter'] + [col for col in stdofavr_BetweenSubjectWconf .columns if metric in col]
    columns_to_extract_with1 = ['subject'] + [col for col in stdofavr_WithinSubjectWconf .columns if metric in col]
    
    statics_bet1 = stdofavr_BetweenSubjectWconf[columns_to_extract_bet1]
    statics_with1 = stdofavr_WithinSubjectWconf[columns_to_extract_with1]

    columns_to_extract_bet = ['iter'] + [col for col in stdofavr_BetweenSubject1.columns if metric in col]
    columns_to_extract_with = ['subject'] + [col for col in stdofavr_WithinSubject1.columns if metric in col]
    
    statics_bet = stdofavr_BetweenSubject1[columns_to_extract_bet]
    statics_with = stdofavr_WithinSubject1[columns_to_extract_with]
    # print(statics_bet)
    # # Calculate standard deviations
    # std_devs_bet = statics_bet.drop(['iter'], axis=1).std()
    # std_devs_with = statics_with.drop(['subject'], axis=1).std()
    
    # statics_bet = pd.concat([statics_bet, pd.DataFrame([std_devs_bet], columns=std_devs_bet.index)], ignore_index=True)
    # statics_bet.at[statics_bet.index[-1], 'iter'] = 'overall_std'

    # statics_with = pd.concat([statics_with, pd.DataFrame([std_devs_with], columns=std_devs_with.index)], ignore_index=True)
    # statics_with.at[statics_with.index[-1], 'subject'] = 'overall_std'
    
    # Melt the DataFrames
    bet_melted1 = statics_bet1.melt(id_vars=['iter'], var_name='metric', value_name='value')
    with_melted1 = statics_with1.melt(id_vars=['subject'], var_name='metric', value_name='value')

    bet_melted = statics_bet.melt(id_vars=['iter'], var_name='metric', value_name='value')
    with_melted = statics_with.melt(id_vars=['subject'], var_name='metric', value_name='value')   
    # Plot box plots for Between Subject
    fig.add_trace(
        go.Box(
            x=bet_melted1['metric'], 
            y=bet_melted1['value'],
            showlegend=False,
            name=f"Between Subject {metric.capitalize()}",
            boxpoints='all',  # Add stripplot-like points
            jitter=0.3,
            pointpos=0,
            marker=dict(
                        color='rgb(34,139,34)'),
            width=0.2
        ),
        row=i + 1, col=1
    )
        # Plot box plots for Between Subject
    fig.add_trace(
        go.Box(
            x=bet_melted['metric'], 
            y=bet_melted['value'],
            showlegend=False,
            name=f"Between Subject {metric.capitalize()}",
            boxpoints='all',  # Add stripplot-like points
            jitter=0.3,
            pointpos=0,
            marker=dict(
                        color='rgb(34,139,34)'),
            width=0.2
        ),
        row=i + 1, col=1
    )
    # Plot box plots for Within Subject
    fig.add_trace(
        go.Box(
            x=with_melted1['metric'], 
            y=with_melted1['value'],
            showlegend=False,
            width=0.2,  # Smaller box width,
            name=f"Within Subject {metric.capitalize()}",
            boxpoints='suspectedoutliers', # only suspected outliers
            jitter=0.3,
            pointpos=0.1,
            marker=dict(
                        color='rgb(8,81,156)',
                        outliercolor='rgb(8,81,156)',
                        line=dict(
                            outliercolor='rgb(8,81,156)',
                            outlierwidth=2)),
            
            text=with_melted1['subject'],  # Hover text showing subject ID
            hovertemplate=(
                "<b>Subject:</b> %{text}<br>"  # Display subject ID
                "<b>Metric:</b> %{x}<br>"      # Display metric
                "<b>Value:</b> %{y}<br>"       # Display value
                "<extra></extra>"              # Remove default extra info
        )
        ),
        row=i + 1, col=1
    )
        # Plot box plots for Within Subject
    fig.add_trace(
        go.Box(
            x=with_melted['metric'], 
            y=with_melted['value'],
            showlegend=False,
            width=0.2,  # Smaller box width,
            name=f"Within Subject {metric.capitalize()}",
            boxpoints='suspectedoutliers', # only suspected outliers
            jitter=0.3,
            pointpos=0.1,
            marker=dict(
                        color='rgb(8,81,156)',
                        outliercolor='rgb(8,81,156)',
                        line=dict(
                            outliercolor='rgb(8,81,156)',
                            outlierwidth=2)),
            
            text=with_melted['subject'],  # Hover text showing subject ID
            hovertemplate=(
                "<b>Subject:</b> %{text}<br>"  # Display subject ID
                "<b>Metric:</b> %{x}<br>"      # Display metric
                "<b>Value:</b> %{y}<br>"       # Display value
                "<extra></extra>"              # Remove default extra info
        )
        ),
        row=i + 1, col=1
    )

# Update layout
fig.update_layout(
    title="Graph metrics with Anatomical and Numerical Variability",
    height=300 * len(metrics),
    showlegend=True,
    legend_title="Legend",
    margin=dict(l=50, r=50, t=50, b=100)
)

# Iterate through rows to remove x-axis tick labels for all subplots
for row in range(1, len(metrics) + 1):
    fig.update_xaxes(showticklabels=False, row=row, col=1)

fig.update_xaxes(
    title_text="Anatomical Variability with confounds" + "&nbsp;" *40 +  "Anatomical Variability No confounds" + "&nbsp;" *40  + "Numerical Variability with confounds" + "&nbsp;" *40 + "Numerical Variability No confounds",
    row=len(metrics),
    col=1
)
# Remove annotations for avr_bet
fig.show()

In [ ]:
globalMettric_WithinSubject = stdofavr_WithinSubject[
    (stdofavr_WithinSubject['session'] == '1') & 
    (stdofavr_WithinSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition',	'degree_(numericalVar)','betweeness_(numericalVar)','clusteringcoef_(numericalVar)','eigenvec_(numericalVar)'])


globalMettric_BetweenSubject= stdofavr_BetweenSubject[
    (stdofavr_BetweenSubject['session'] == '1') & 
    (stdofavr_BetweenSubject['acquisition'] == 'acq-RL')].drop(columns=['session', 'acquisition','degree_(AnatomicalVar)','betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)','eigenvec_(AnatomicalVar)'])


globalMettric_WithinSubjectWconf = df_stdofavr_WithinSubject[
    (df_stdofavr_WithinSubject['session'] == '1') & 
    (df_stdofavr_WithinSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition',	'degree_(numericalVar)','betweeness_(numericalVar)','clusteringcoef_(numericalVar)','eigenvec_(numericalVar)'])


globalMettric_BetweenSubjectWconf= df_stdofavr_BetweenSubject[
    (df_stdofavr_BetweenSubject['session'] == '1') & 
    (df_stdofavr_BetweenSubject['acquisition'] == 'acq-RL')].drop(columns=['session', 'acquisition','degree_(AnatomicalVar)','betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)','eigenvec_(AnatomicalVar)'])
globalMettric_WithinSubjectWconf = globalMettric_WithinSubjectWconf.rename(columns={'smallworldness(numericalVar)': 'smallworldness(numericalVarW)', 'avg_shortestPathLength(numericalVar)': 'avg_shortestPathLength(numericalVarW)'   })
globalMettric_BetweenSubjectWconf = globalMettric_BetweenSubjectWconf.rename(columns={'smallworldness_(AnatomicalVar)': 'smallworldness_(AnatomicalVarW)', 'avg_shortestPathLength_(AnatomicalVar)': 'avg_shortestPathLength_(AnatomicalVarW)' })


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots

# List of metrics
metrics = ['smallworldness', 'avg_shortestPathLength']

# Create subplots
fig = make_subplots(
    rows=len(metrics), 
    cols=1,  
    vertical_spacing=0.05,     
    subplot_titles=[
        f"Small worldness with anatomical mean = {avr_BetweenSubject['smallworldness_(AnatomicalVar)'].mean()}",
        f"Avrg shortest path with anatomical mean = {avr_BetweenSubject['avg_shortestPathLength_(AnatomicalVar)'].mean()}"
    ]
)



for i, metric in enumerate(metrics):

        # Extract columns for each metric
    columns_to_extract_bet1 = ['iter'] + [col for col in globalMettric_BetweenSubjectWconf.columns if metric in col]
    columns_to_extract_with1 = ['subject'] + [col for col in globalMettric_WithinSubjectWconf.columns if metric in col]
    
    statics_bet1 = globalMettric_BetweenSubjectWconf[columns_to_extract_bet1]
    statics_with1 = globalMettric_WithinSubjectWconf[columns_to_extract_with1]
    # Extract columns for each metric
    columns_to_extract_bet = ['iter'] + [col for col in globalMettric_BetweenSubject.columns if metric in col]
    columns_to_extract_with = ['subject'] + [col for col in globalMettric_WithinSubject.columns if metric in col]
    
    statics_bet = globalMettric_BetweenSubject[columns_to_extract_bet]
    statics_with = globalMettric_WithinSubject[columns_to_extract_with]
    
    # # Calculate standard deviations
    # std_devs_bet = statics_bet.drop(['iter'], axis=1).std()
    # std_devs_with = statics_with.drop(['subject'], axis=1).std()
    
    # statics_bet = pd.concat([statics_bet, pd.DataFrame([std_devs_bet], columns=std_devs_bet.index)], ignore_index=True)
    # statics_bet.at[statics_bet.index[-1], 'iter'] = 'overall_std'

    # statics_with = pd.concat([statics_with, pd.DataFrame([std_devs_with], columns=std_devs_with.index)], ignore_index=True)
    # statics_with.at[statics_with.index[-1], 'subject'] = 'overall_std'
    
    # Melt the DataFrames
    bet_melted1 = statics_bet1.melt(id_vars=['iter'], var_name='metric', value_name='value')
    with_melted1 = statics_with1.melt(id_vars=['subject'], var_name='metric', value_name='value')
    # Melt the DataFrames
    bet_melted = statics_bet.melt(id_vars=['iter'], var_name='metric', value_name='value')
    with_melted = statics_with.melt(id_vars=['subject'], var_name='metric', value_name='value')
    # Plot box plots for Between Subject
    fig.add_trace(
        go.Box(
            x=bet_melted1['metric'], 
            y=bet_melted1['value'],
            showlegend=False,
            name=f"Between Subject With confounds{metric.capitalize()}",
            boxpoints='all',  # Add stripplot-like points
            jitter=0.3,
            pointpos=0,
            marker=dict(
                        color='rgb(34,139,34)'),
            width=0.2
        ),
        row=i + 1, col=1
    )
     # Plot box plots for Between Subject
    fig.add_trace(
        go.Box(
            x=bet_melted['metric'], 
            y=bet_melted['value'],
            showlegend=False,
            name=f"Between Subject {metric.capitalize()}",
            boxpoints='all',  # Add stripplot-like points
            jitter=0.3,
            pointpos=0,
            marker=dict(
                        color='rgb(34,139,34)'),
            width=0.2
        ),
        row=i + 1, col=1
    )   
    # Plot box plots for Within Subject
    fig.add_trace(
        go.Box(
            x=with_melted1['metric'], 
            y=with_melted1['value'],
            showlegend=False,
            width=0.2,  # Smaller box width,
            name=f"Within Subject With confounds{metric.capitalize()}",
            boxpoints='suspectedoutliers', # only suspected outliers
            jitter=0.3,
            pointpos=0.1,
            marker=dict(
                        color='rgb(8,81,156)',
                        outliercolor='rgb(8,81,156)',
                        line=dict(
                            outliercolor='rgb(8,81,156)',
                            outlierwidth=2)),
            
            text=with_melted1['subject'],  # Hover text showing subject ID
            hovertemplate=(
                "<b>Subject:</b> %{text}<br>"  # Display subject ID
                "<b>Metric:</b> %{x}<br>"      # Display metric
                "<b>Value:</b> %{y}<br>"       # Display value
                "<extra></extra>"              # Remove default extra info
        )
        ),
        row=i + 1, col=1
    )
    # Plot box plots for Within Subject
    fig.add_trace(
        go.Box(
            x=with_melted['metric'], 
            y=with_melted['value'],
            showlegend=False,
            width=0.2,  # Smaller box width,
            name=f"Within Subject {metric.capitalize()}",
            boxpoints='suspectedoutliers', # only suspected outliers
            jitter=0.3,
            pointpos=0.1,
            marker=dict(
                        color='rgb(8,81,156)',
                        outliercolor='rgb(8,81,156)',
                        line=dict(
                            outliercolor='rgb(8,81,156)',
                            outlierwidth=2)),
            
            text=with_melted['subject'],  # Hover text showing subject ID
            hovertemplate=(
                "<b>Subject:</b> %{text}<br>"  # Display subject ID
                "<b>Metric:</b> %{x}<br>"      # Display metric
                "<b>Value:</b> %{y}<br>"       # Display value
                "<extra></extra>"              # Remove default extra info
        )
        ),
        row=i + 1, col=1
    )

# Update layout
fig.update_layout(
    title="Metrics with Anatomical and Numerical Variability",
    height=300 * len(metrics),
    showlegend=True,
    legend_title="Legend",
    margin=dict(l=50, r=50, t=50, b=100)
)

# Iterate through rows to remove x-axis tick labels for all subplots
for row in range(1, len(metrics) + 1):
    fig.update_xaxes(showticklabels=False, row=row, col=1)

fig.update_xaxes(
    title_text="Anatomical Variability with confounds" + "&nbsp;" *40 +  "Anatomical Variability No confounds" + "&nbsp;" *40  + "Numerical Variability with confounds" + "&nbsp;" *40 + "Numerical Variability No confounds",
    row=len(metrics),
    col=1
)

# Remove annotations for avr_bet
fig.show()

In [ ]:
column_names = ['subject_id', 'session','acquisition','rep', 'repp', 'region_label', 'dice_coef']

# Load the CSV file into a DataFrame with specified column names
df_ROI_atlasinT1W = pd.read_csv('/home/ubuntu/Desktop/Thesis/overlap/batchhc/dice_reg_inverse.csv', header=None, names=column_names)
df_ROI_atlasinT1W = df_ROI_atlasinT1W.drop(0).reset_index(drop=True)
df_ROI_atlasinT1W
df_ROI_atlasinT1W['region_label'] = df_ROI_atlasinT1W['region_label'].astype('int64')
df_ROI_atlasinT1W['session'] = df_ROI_atlasinT1W['session'].astype('int64')



# Load the CSV file into a DataFrame with specified column names
df_ROI_atlasinT1W2Bold = pd.read_csv('/home/ubuntu/Desktop/Thesis/overlap/batchhc/dice_coreg_inverse.csv', header=None, names=column_names)
# df1 = pd.read_csv('/home/ubuntu/Desktop/Thesis/overlap/batch2/dice_coreg_inverseadd.csv', header=None, names=column_names)
# Drop the first row (column names) from the second DataFrame
# df1 = df1[1:].reset_index(drop=True)
# df_ROI_atlasinT1W2Bold=pd.concat([df, df1], ignore_index=True)
df_ROI_atlasinT1W2Bold = df_ROI_atlasinT1W2Bold.drop(0).reset_index(drop=True)
df_ROI_atlasinT1W2Bold['region_label'] = df_ROI_atlasinT1W2Bold['region_label'].astype('int64')
df_ROI_atlasinT1W2Bold['session'] = df_ROI_atlasinT1W2Bold['session'].astype('int64')



In [ ]:
import numpy as np

def create_stackmatrix_OfDice(result_df):
    num_sub =len(result_df['subject_id'].unique()) # number of subjects
    num_regions = 100  # number of regions
    num_MCApairs = 45  # number of dice coefficient combinations
    # Initialize stack matrix to hold the data for each subject
    stack_matrix = np.empty((num_sub, num_regions, num_MCApairs))
    index_to_subject = {}

    for index, sub in enumerate(result_df['subject_id'].unique()[:num_sub]):
        Dice_coef = np.zeros((num_regions, num_MCApairs))
        # Store the mapping of index to subject ID
        index_to_subject[index] = sub  
        # Filter the DataFrame for the current subject
        subject_data = result_df[result_df['subject_id'] == sub]
        # print(sub, subject_data)
        for region in range(1, num_regions + 1):
            # Get the dice coefficients for the current region and subject
            region_data = subject_data[subject_data['region_label'] == region]['dice_coef'].values.tolist()[0]
            # Ensure we have the expected number of coefficients (45) for each region
            if len(region_data) == num_MCApairs:
                Dice_coef[region - 1, :] = region_data
            else:
                print(f"Warning: Region {region} for subject {sub} has {len(region_data)} values instead of {num_MCApairs}.")
        
        # Assign the computed Dice_coef matrix for the current subject to stack_matrix
        stack_matrix[index] = Dice_coef
    return stack_matrix,index_to_subject


In [ ]:
result_df_atlasinT1W=df_ROI_atlasinT1W[(df_ROI_atlasinT1W['session']==1) & (df_ROI_atlasinT1W['acquisition']=='acq-RL')] 
result_df_atlasinT1W=result_df_atlasinT1W.drop(['session', 'acquisition'], axis=1)
# Group the DataFrame by subject ID and aggregate the region_label and dice_coef into lists
result_df_atlasinT1W = result_df_atlasinT1W.groupby(['subject_id', 'region_label']).agg({

    'dice_coef': list
}).reset_index()

result_df_ROI_atlasinT1W2Bold=df_ROI_atlasinT1W2Bold[(df_ROI_atlasinT1W2Bold['session']==1) & (df_ROI_atlasinT1W2Bold['acquisition']=='acq-RL')] 
# Group the DataFrame by subject ID and aggregate the region_label and dice_coef into lists
result_df_ROI_atlasinT1W2Bold = result_df_ROI_atlasinT1W2Bold.groupby(['subject_id', 'region_label']).agg({

    'dice_coef': list
}).reset_index()


In [ ]:
import nibabel as nib
import numpy as np
import pandas as pd

# Load the Schaefer atlas NIfTI file
atlas_path = "/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.nii.gz"
atlas_img = nib.load(atlas_path)
atlas_data = atlas_img.get_fdata()

# Load the TSV file with atlas labels
tsv_file_path = "/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.tsv"
atlas_df = pd.read_csv(tsv_file_path, delimiter='\t')

# Extract labels
atlas_labels = atlas_df['name'].tolist()

# Get unique region labels (excluding background if label = 0)
unique_labels = np.unique(atlas_data)
unique_labels = unique_labels[unique_labels > 0]  # Remove background (0)

# Compute voxel count per region
region_sizes = {int(label): np.sum(atlas_data == label) for label in unique_labels}

# Get voxel size in mm³ (multiply dimensions)
voxel_volume = np.prod(atlas_img.header.get_zooms())  # Voxel size in mm³

# Convert voxel count to mm³
region_sizes_mm3 = {label: count * voxel_volume for label, count in region_sizes.items()}

# Create a DataFrame for clarity
region_size_df = pd.DataFrame({
    'Region': atlas_labels,
    'Label': list(region_sizes.keys()),
    'Voxel_Count': list(region_sizes.values()),
    'Size_mm3': list(region_sizes_mm3.values())
})

# Display region sizes
print(region_size_df)


# Sort regions by size (largest to smallest)
region_size_df_sorted = region_size_df.sort_values(by="Size_mm3", ascending=False)

# Display sorted region sizes
print(region_size_df_sorted)



In [ ]:
#select one acusition for subject
stack_matrix_T1,indexT=create_stackmatrix_OfDice(result_df_atlasinT1W)
stack_matrix_bold,indexB=create_stackmatrix_OfDice(result_df_ROI_atlasinT1W2Bold)

Numerical_min_T1=  np.min(stack_matrix_T1,axis=2)
numerical_min_bold=np.min(stack_matrix_bold,axis=2)
min_T1=np.mean(Numerical_min_T1,axis=0)
min_bold=np.mean(numerical_min_bold,axis=0)


In [ ]:
print(stack_matrix_T1.shape)

# Assuming stack_matrix is your (112, 100, 45) matrix
average_per_subject = np.mean(stack_matrix_T1, axis=(1, 2))  # Shape will be (112,)
top_min_indices = np.argsort(average_per_subject)[:10]
print(top_min_indices)
average_per_subjectb = np.mean(stack_matrix_bold, axis=(1, 2))  # Shape will be (112,)
btop_min_indices = np.argsort(average_per_subjectb)[:10]
print(btop_min_indices)

filtered_indexT = {k: v for k, v in indexT.items() if k not in top_min_indices}
filtered_indexB = {k: v for k, v in indexB.items() if k not in btop_min_indices}
print((filtered_indexB))
print((indexB))

stack_matrix_T1_filtered = np.delete(stack_matrix_T1, top_min_indices, axis=0)
stack_matrix_bold_filtered = np.delete(stack_matrix_bold, btop_min_indices, axis=0)
Numerical_min_T1_filtered=  np.min(stack_matrix_T1_filtered,axis=2)
numerical_min_bold_filtered=np.min(stack_matrix_bold_filtered,axis=2)
min_T1_filtered=np.mean(Numerical_min_T1_filtered,axis=0)
min_bold_filtered=np.mean(numerical_min_bold_filtered,axis=0)


In [ ]:
print(stack_matrix_bold.shape)

In [ ]:
from nilearn import plotting, datasets, image
import numpy as np
import nibabel as nib
def Return_Metric_Img(metric_matrix):
    # Load an example atlas, e.g., MNI152 template
    # atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100, resolution_mm=2)

    # # Assume `metric_values` is a list or array of values for the 100 regions
    metric_values = metric_matrix # Replace with your actual metric values

    # # Load the brain atlas and get the atlas regions
    # atlas_img = nib.load(atlas.maps)
    # Load your atlas NIfTI file
    atlas='/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.nii.gz'
    # Define the path to your TSV file
    tsv_file_path = "/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.tsv"
    atlas_img=nib.load(atlas)
    atlas_data = atlas_img.get_fdata()
    # Create an empty image with the same shape as the atlas
    metric_img_data = np.zeros(atlas_img.shape)

    # Assign metric values to the atlas regions
    for i, region in enumerate(np.unique(atlas_img.get_fdata())):
        if region != 0:  # Ignore the background (region 0)
            metric_img_data[atlas_img.get_fdata() == region] = metric_values[i-1]  # i-1 to skip background

    # Create a new NIfTI image with metric values
    metric_img = nib.Nifti1Image(metric_img_data, atlas_img.affine)
    return metric_img


In [ ]:
from nilearn import plotting, datasets
import matplotlib.pyplot as plt
import numpy as np
import textwrap


# Assuming you have two different metrics as images
metric_img1 = Return_Metric_Img(min_T1)
metric_img2 = Return_Metric_Img(min_bold)
# vmin = min(np.min(min_T1), np.min(min_bold))
vmin=0.85
vmax = max(np.max(min_T1), np.max(min_bold))

print(np.min(min_T1), np.min(min_bold), np.max(min_T1), np.max(min_bold))
# Create a subplot with two figures (1 row, 2 columns)
fig, axes = plt.subplots(2, 1, figsize=(20, 12))

# Set color range and colormap
vmax = 1
cmap = "Reds_r"

# Plot the first metric with an individual colorbar
plotting.plot_stat_map(
    metric_img1,
    bg_img=datasets.load_mni152_template(),
    display_mode='ortho',
    cut_coords=(10, 0, 0),
    title='average dice scores of Atlas_InT1W ',
    cmap=cmap,
    axes=axes[0],
    vmin=0.85, #np.min(min_T1),
    vmax=vmax,
    colorbar=False
)

# Plot the second metric with an individual colorbar
plotting.plot_stat_map(
    metric_img2,
    bg_img=datasets.load_mni152_template(),
    display_mode='ortho',
    cut_coords=(10, 0, 0),
    title='average dice scores of Atlas_InBold',
    cmap=cmap,
    axes=axes[1],
    vmin=0.85, #np.min(min_bold),
    vmax=vmax,
    colorbar=False
)
# Add a common colorbar for both plots with extended tick marks
cbar_ax = fig.add_axes([1, 0.25, 0.02, 0.5])
norm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

# Generate custom tick marks across the color range
ticks = np.linspace(vmin, vmax, 15)  # Adjust the number of ticks here for finer granularity
cbar = fig.colorbar(norm, cax=cbar_ax, label='Dice scores', ticks=ticks)
# Show the plots
plt.tight_layout()

# Define the text and wrap it to a specific width
text = ("Figure 1: Dice scores were computed for each region across each pair of MCA runs. The minimum Dice scores were identified and then the averages were calculated across subjects for each region. The figure shows the average Dice scores for the registered atlas in subject space (top) and BOLD space (bottom).")
wrapped_text = "\n".join(textwrap.wrap(text, width=180))  # Adjust the width as needed

# Add the wrapped text to the figure
plt.figtext(0.5, -0.05, wrapped_text, ha="center", fontsize=15,  wrap=True)
plt.show()


In [ ]:
from nilearn import plotting, datasets
import matplotlib.pyplot as plt
import numpy as np
import textwrap


# Assuming you have two different metrics as images
metric_img1_filtered = Return_Metric_Img(min_T1_filtered)
metric_img2_filtered = Return_Metric_Img(min_bold_filtered)
# vmin = min(np.min(min_T1_filtered), np.min(min_bold_filtered))
# vmax = max(np.max(min_T1_filtered), np.max(min_bold_filtered))
# vmin = min(np.min(min_T1), np.min(min_bold))
vmin=0.89
vmax = max(np.max(min_T1), np.max(min_bold))
print(np.min(min_T1_filtered), np.min(min_bold_filtered), np.max(min_T1_filtered), np.max(min_bold_filtered))
# Create a subplot with two figures (1 row, 2 columns)
fig, axes = plt.subplots(2, 1, figsize=(20, 12))

# Set color range and colormap
vmax = 1
cmap = "Reds_r"

# Plot the first metric with an individual colorbar
plotting.plot_stat_map(
    metric_img1_filtered,
    bg_img=datasets.load_mni152_template(),
    display_mode='ortho',
    cut_coords=(10, 0, 0),
    title='average dice scores of Atlas_InT1W ',
    cmap=cmap,
    axes=axes[0],
    vmin=0.89,
    vmax=vmax,
    colorbar=False
)

# Plot the second metric with an individual colorbar
plotting.plot_stat_map(
    metric_img2_filtered,
    bg_img=datasets.load_mni152_template(),
    display_mode='ortho',
    cut_coords=(10, 0, 0),
    title='average dice scores of Atlas_InBold',
    cmap=cmap,
    axes=axes[1],
    vmin=0.89,
    vmax=vmax,
    colorbar=False
)
# Add a common colorbar for both plots with extended tick marks
cbar_ax = fig.add_axes([1, 0.25, 0.02, 0.5])
norm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

# Generate custom tick marks across the color range
ticks = np.linspace(vmin, vmax, 15)  # Adjust the number of ticks here for finer granularity
cbar = fig.colorbar(norm, cax=cbar_ax, label='Dice scores', ticks=ticks)
# Show the plots
plt.tight_layout()

# Define the text and wrap it to a specific width
text = ("Figure 1: Dice scores were computed for each region across each pair of MCA runs. The minimum Dice scores were identified and then the averages were calculated across subjects for each region. The figure shows the average Dice scores for the registered atlas in subject space (top) and BOLD space (bottom).")
wrapped_text = "\n".join(textwrap.wrap(text, width=180))  # Adjust the width as needed

# Add the wrapped text to the figure
plt.figtext(0.5, -0.05, wrapped_text, ha="center", fontsize=15,  wrap=True)
plt.show()


$$
Numerical Variability:  \forall r \in \left[ 1 \hspace{0.1cm}, R \right],  \hspace{0.5cm} 
\Delta_{r} = \text{mean}_{s \in \left[ 1 \hspace{0.1cm}, S \right]} \left( \min_{(p,p^{'}),(p^{''},p^{'''}) \in \left[ 1 \hspace{0.1cm}, P \right]} 
\hspace{0.2cm} D \left( V_{r,s,(p,p^{'})}, V_{r,s,(p^{''},p^{'''})} \right) \right)
\newline

\hspace{0.5cm}R:\text{Number of Regions}, \hspace{0.1cm} S: \text{Number of Subjects}, \hspace{0.2cm} \text{and} \hspace{0.2cm} P: \text{Number of MCA pairs}\,(p,p^{'})


$$

In [ ]:
#Sorting based on region size
avrofMCApairs_T1=np.mean(stack_matrix_T1,axis=2)
avrofMCApairs_bold=np.mean(stack_matrix_bold,axis=2)
print(avrofMCApairs_T1.shape)

# Step 1: Get sorted region indices based on size
sorted_indices = region_size_df_sorted['Label'].index.values  # Get indices after sorting
sorted_region=region_size_df_sorted['Region'].values
print((sorted_region[0]))
# Step 2: Apply sorting to the columns of the matrices
sorted_avrofMCApairs_T1 = avrofMCApairs_T1[:, sorted_indices]  # Reorder columns
sorted_avrofMCApairs_bold = avrofMCApairs_bold[:, sorted_indices]  # Reorder columns



In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = min(sorted_avrofMCApairs_T1.min(),sorted_avrofMCApairs_bold.min())
vmax = max(sorted_avrofMCApairs_T1.max(), sorted_avrofMCApairs_bold.max())
# Convert index dictionaries to ordered lists of subject IDs
indexT_labels = [indexT[i] for i in range(len(indexT))]  # Extract subject IDs from indexT dictionary
indexB_labels = [indexB[i] for i in range(len(indexB))]  # Extract subject IDs from indexB dictionary
sorted_region=region_size_df_sorted['Region'].values

# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sorted_avrofMCApairs_T1,
        y=indexT_labels,  # Use extracted subject IDs for BOLD
        x=sorted_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="Average", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sorted_avrofMCApairs_bold,
        y=indexB_labels,  # Use extracted subject IDs for BOLD
        x=sorted_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of average of MCApairs_T1 and average of MCApairs_bold (Shared Color Map)",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()


In [ ]:
Remove 5 worse subjects

In [ ]:
#Sorting after removing worse subjects based on region size
avrofMCApairs_T1_filtered=np.mean(stack_matrix_T1_filtered,axis=2)
avrofMCApairs_bold_filtered=np.mean(stack_matrix_bold_filtered,axis=2)

# Step 1: Get sorted region indices based on size
sorted_indices = region_size_df_sorted['Label'].index.values  # Get indices after sorting
sorted_region=region_size_df_sorted['Region'].values
print((sorted_region[1]))
# Step 2: Apply sorting to the columns of the matrices

sorted_avrofMCApairs_T1_filtered = avrofMCApairs_T1_filtered[:, sorted_indices]  # Reorder columns
sorted_avrofMCApairs_bold_filtered= avrofMCApairs_bold_filtered[:, sorted_indices]  # Reorder columns

# Now sorted_avrofMCApairs_T1 and sorted_avrofMCApairs_bold have regions sorted by increasing size



In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = min(sorted_avrofMCApairs_T1_filtered.min(), sorted_avrofMCApairs_bold_filtered.min())
vmax = max(sorted_avrofMCApairs_T1_filtered.max(), sorted_avrofMCApairs_bold_filtered.max())
# Convert index dictionaries to ordered lists of subject IDs
indexT_labels_filtered = [filtered_indexT[i] for i in filtered_indexT.keys()]  # Extract subject IDs from indexT dictionary
indexB_labels_filtered = [filtered_indexB[i] for i in filtered_indexB.keys()]  # Extract subject IDs from indexB dictionary
sorted_region=region_size_df_sorted['Region'].values

# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sorted_avrofMCApairs_T1_filtered,
        y=indexT_labels_filtered,
        x=sorted_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="rainbow_r", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sorted_avrofMCApairs_bold_filtered,
        y=indexB_labels_filtered,
        x=sorted_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of average dice scores of MCApairs_T1 and average of MCApairs_bold across sorted region by size after removing 10 worse subjects from both T1 and Bold",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()



In [ ]:
# import plotly.subplots as sp
# import plotly.graph_objects as go
# import numpy as np

# # Determine shared color map limits
# vmin = min(sorted_avrofMCApairs_T1_filtered.min(), sorted_avrofMCApairs_bold_filtered.min())
# vmax = max(sorted_avrofMCApairs_T1_filtered.max(), sorted_avrofMCApairs_bold_filtered.max())
# # Convert index dictionaries to ordered lists of subject IDs
# indexT_labels_filtered = [filtered_indexT[i] for i in filtered_indexT.keys()]  # Extract subject IDs from indexT dictionary
# indexB_labels_filtered = [filtered_indexB[i] for i in filtered_indexB.keys()]  # Extract subject IDs from indexB dictionary
# sorted_region=region_size_df_sorted['Region'].values

# # Create subplots
# fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# # Add first heatmap
# fig.add_trace(
#     go.Heatmap(
#         z=sorted_avrofMCApairs_T1_filtered,
#         y=indexT_labels_filtered,
#         x=sorted_region,
#         colorscale='rainbow_r',
#         zmin=vmin,
#         zmax=vmax,
#         colorbar=dict(title="rainbow_r", len=0.8),
#     ),
#     row=1,
#     col=1
# )

# # Add second heatmap (share the same color scale)
# fig.add_trace(
#     go.Heatmap(
#         z=sorted_avrofMCApairs_bold_filtered,
#         y=indexB_labels_filtered,
#         x=sorted_region,
#         colorscale='rainbow_r',
#         zmin=vmin,
#         zmax=vmax,
#         showscale=False,  # Hide extra colorbar
#     ),
#     row=1,
#     col=2
# )

# # Update y-axis labels
# fig.update_yaxes(title_text="Subject ID", row=1, col=1)
# fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# # Update layout
# fig.update_layout(
#     height=1000, width=2000,
#     title_text="Heatmaps of average dice scores of MCApairs_T1 and average of MCApairs_bold across sorted region by size after removing 10 worse subjects from both T1 and Bold",
#     coloraxis_colorbar=dict(len=0.8)
# )

# fig.show()



In [ ]:
#Sorting based on region size and average across subjects values
import numpy as np

# Compute mean across subjects (axis=0)
mean_values = np.mean(sorted_avrofMCApairs_T1, axis=0)  
mean_valuesb = np.mean(sorted_avrofMCApairs_bold, axis=0)  

# Get sorted indices based on mean values
sortedavr_indices = np.argsort(mean_values).astype(int)  
sortedavr_indicesb = np.argsort(mean_valuesb).astype(int)  

# Convert region labels to NumPy array and reorder
sorted_region = np.array(region_size_df_sorted['Region'].values)  
sortedavr_region = sorted_region[sortedavr_indices]  
sortedavr_regionb = sorted_region[sortedavr_indices]  

# Reorder matrix columns (regions)
sortedavr_avrofMCApairs_T1= sorted_avrofMCApairs_T1[:, sortedavr_indices]
sortedavr_avrofMCApairs_bold_T1= sorted_avrofMCApairs_bold[:, sortedavr_indices]
sortedavr_avrofMCApairs_bold= sorted_avrofMCApairs_bold[:, sortedavr_indicesb]

# Compute mean across regions for each subject (axis=1)
Smean_values = np.mean(sortedavr_avrofMCApairs_T1, axis=1)  
Smean_valuesb = np.mean(sortedavr_avrofMCApairs_bold, axis=1)  

# Get sorted indices for subjects (convert to integer type)
Ssortedavr_indices = np.argsort(Smean_values).astype(int)  
Ssortedavr_indicesb = np.argsort(Smean_valuesb).astype(int)  

# Convert subject labels to NumPy array
indexT_labels= np.array([indexT[i] for i in indexT.keys()])  
indexB_labels= np.array([indexB[i] for i in indexB.keys()])  

# Reorder subject labels **(For reference, NOT for indexing)**
sorted_indexT_labels = indexT_labels[Ssortedavr_indices]  
sorted_indexB_labels= indexB_labels[Ssortedavr_indicesb]  

# sortedavr_stdofMCApairs_T1 = sortedavr_stdofMCApairs_T1[Ssortedavr_indices, :]  
# sortedavr_stdofMCApairs_bold= sortedavr_stdofMCApairs_bold[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_T1= sortedavr_avrofMCApairs_T1[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_bold_T1= sortedavr_avrofMCApairs_bold_T1[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_bold= sortedavr_avrofMCApairs_bold[Ssortedavr_indicesb, :]  



In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = 0.85 # min(sortedavr_avrofMCApairs_T1.min(), sortedavr_avrofMCApairs_bold.min())
vmax = max(sortedavr_avrofMCApairs_T1.max(), sortedavr_avrofMCApairs_bold_T1.max())


# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_T1,
        y=sorted_indexT_labels,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="rainbow_r", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_bold_T1,
        y=sorted_indexT_labels,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of the AVerage of MCA pairs_T1 and Average of MCApairs_bold sorted based on the average dice scores across subject and across regions of T1 ",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()





In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = 0.85 # min(sortedavr_avrofMCApairs_T1.min(), sortedavr_avrofMCApairs_bold.min())
vmax = max(sortedavr_avrofMCApairs_T1.max(), sortedavr_avrofMCApairs_bold.max())


# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_T1,
        y=sorted_indexT_labels,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="rainbow_r", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_bold,
        y=sorted_indexB_labels,
        x=sortedavr_regionb,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of the AVerage of MCA pairs_T1 and Average of MCApairs_bold sorted based on the average dice scores across subject and across regions of T1 and bold separatedly ",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()



In [ ]:
import numpy as np

# Compute mean across subjects (axis=0)
mean_values = np.mean(sorted_avrofMCApairs_T1_filtered, axis=0)  
mean_valuesb = np.mean(sorted_avrofMCApairs_bold_filtered, axis=0)  

# Get sorted indices based on mean values
sortedavr_indices = np.argsort(mean_values).astype(int)  
sortedavr_indicesb = np.argsort(mean_valuesb).astype(int)  

# Convert region labels to NumPy array and reorder
sorted_region = np.array(region_size_df_sorted['Region'].values)  
sortedavr_region = sorted_region[sortedavr_indices]  
sortedavr_regionb = sorted_region[sortedavr_indicesb]  

# Reorder matrix columns (regions)

sortedavr_avrofMCApairs_T1_filtered = sorted_avrofMCApairs_T1_filtered[:, sortedavr_indices]
sortedavr_avrofMCApairs_bold_T1_filtered = sorted_avrofMCApairs_bold_filtered[:, sortedavr_indices]
sortedavr_avrofMCApairs_bold_filtered = sorted_avrofMCApairs_bold_filtered[:, sortedavr_indicesb]

# Compute mean across regions for each subject (axis=1)
Smean_values = np.mean(sortedavr_avrofMCApairs_T1_filtered, axis=1)  
Smean_valuesb = np.mean(sortedavr_avrofMCApairs_bold_filtered, axis=1)  

# Get sorted indices for subjects (convert to integer type)
Ssortedavr_indices = np.argsort(Smean_values).astype(int)  
Ssortedavr_indicesb = np.argsort(Smean_valuesb).astype(int)  

# Convert subject labels to NumPy array
indexT_labels_filtered = np.array([filtered_indexT[i] for i in filtered_indexT.keys()])  
indexB_labels_filtered = np.array([filtered_indexB[i] for i in filtered_indexB.keys()])  

# Reorder subject labels **(For reference, NOT for indexing)**
sorted_indexT_labels_filtered = indexT_labels_filtered[Ssortedavr_indices]  
sorted_indexB_labels_filtered = indexB_labels_filtered[Ssortedavr_indicesb]  


sortedavr_avrofMCApairs_T1_filtered = sortedavr_avrofMCApairs_T1_filtered[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_bold_T1_filtered = sortedavr_avrofMCApairs_bold_T1_filtered[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_bold_filtered = sortedavr_avrofMCApairs_bold_filtered[Ssortedavr_indicesb, :]  




In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = 0.85 # min(sortedavr_avrofMCApairs_T1.min(), sortedavr_avrofMCApairs_bold.min())
vmax = max(sortedavr_avrofMCApairs_T1_filtered.max(), sortedavr_avrofMCApairs_bold_T1_filtered.max())


# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_T1_filtered,
        y=sorted_indexT_labels_filtered,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="rainbow_r", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_bold_T1_filtered,
        y=sorted_indexT_labels_filtered,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of the AVerage of MCA pairs_T1 and Average of MCApairs_bold sorted based on the average dice scores across subject and across regions of T1 ",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()


In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = 0.85 #min(sortedavr_avrofMCApairs_T1_filtered.min(), sortedavr_avrofMCApairs_bold_filtered.min())
vmax = max(sortedavr_avrofMCApairs_T1_filtered.max(), sortedavr_avrofMCApairs_bold_filtered.max())


# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_T1_filtered,
        y=sorted_indexT_labels_filtered,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="Average", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_bold_filtered,
        y=sorted_indexB_labels_filtered,
        x=sortedavr_regionb,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of the AVerage of MCA pairs_T1 and Average of MCApairs_bold sorted based on the average dice scores across subject and across regions of T1 and bold separatedly",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()




In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = 0.85 #min(sortedavr_avrofMCApairs_T1_filtered.min(), sortedavr_avrofMCApairs_bold_filtered.min())
vmax = max(sortedavr_avrofMCApairs_T1_filtered.max(), sortedavr_avrofMCApairs_bold_filtered.max())


# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_T1_filtered,
        y=sorted_indexT_labels_filtered,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="Average", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_bold_filtered,
        y=sorted_indexB_labels_filtered,
        x=sortedavr_regionb,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of the AVerage of MCA pairs_T1 and Average of MCApairs_bold sorted based on the average dice scores across subject and across regions of T1 and bold separatedly",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()







In [ ]:
stack_matrix_T11=create_stackmatrix_OfDice(result_df_atlasinT1W1)
stack_matrix_bold1=create_stackmatrix_OfDice(result_df_ROI_atlasinT1W2Bold1)

Numerical_min_T11=  np.min(stack_matrix_T11,axis=2)
numerical_min_bold1=np.min(stack_matrix_bold1,axis=2)
min_T11=np.mean(Numerical_min_T11,axis=0)
min_bold1=np.mean(numerical_min_bold1,axis=0)

<!-- $$
Numerical Variability:  \forall r \in \left[ 1 \hspace{0.1cm}, R \right],  \hspace{0.5cm} 
\Delta_{r} = \text{mean}_{s \in \left[ 1 \hspace{0.1cm}, S \right]} \left( \min_{(p,p^{'}),(p^{''},p^{'''}) \in \left[ 1 \hspace{0.1cm}, P \right]} 
\hspace{0.2cm} D \left( V_{r,s,(p,p^{'})}, V_{r,s,(p^{''},p^{'''})} \right) \right)
\newline
Anatomical Variability:  \forall r \in \left[ 1 \hspace{0.1cm}, R \right], \hspace{0.5cm} \Delta^{'}_{r} = \text{mean}_{(p,p^{'}) \in \left[ 1 \hspace{0.1cm}, P \right]} \left( \min_{(s,s^{'}) \in \left[ 1 \hspace{0.1cm}, S \right]} 
\hspace{0.2cm} D \left( V_{r,s,(p,p^{'})}, V_{r,s^{'},(p,p^{'})} \right) \right)\\
\newline
\hspace{0.5cm}R:\text{Number of Regions}, \hspace{0.1cm} S: \text{Number of Subjects}, \hspace{0.2cm} \text{and} \hspace{0.2cm} P: \text{Number of MCA pairs}\,(p,p^{'})


$$ -->

In [ ]:
#second formula without computing mean for across subject
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Example lists
region_size = list(region_sizes.values())
Dice_value = min_T1   # Your list of 100 numerical values

# Calculate Pearson correlation
correlation, _ = pearsonr(Dice_value, region_size)

# Create scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(region_size, Dice_value, color='blue', label=f'Pearson Correlation: {correlation:.2f}')

# Add labels and title
plt.xlabel('region size')
plt.ylabel('Dice Values of T1 registration')
plt.title('Scatter Plot of Pearson Correlation')

# Show the correlation value on the plot
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#second formula without computing mean for across subject
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Example lists
region_size = list(region_sizes.values())
Dice_value = min_bold   # Your list of 100 numerical values

# Calculate Pearson correlation
correlation, _ = pearsonr(Dice_value, region_size)

# Create scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(region_size, Dice_value, color='blue', label=f'Pearson Correlation: {correlation:.2f}')

# Add labels and title
plt.xlabel('region size')
plt.ylabel('Dice Values of bold registration')
plt.title('Scatter Plot of Pearson Correlation')

# Show the correlation value on the plot
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from nilearn import datasets, image
import numpy as np
from scipy.ndimage import center_of_mass
# Load your atlas NIfTI file
atlas='/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.nii.gz'
# Define the path to your TSV file
tsv_file_path = "/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.tsv"
atlas_img=nib.load(atlas)
atlas_data = atlas_img.get_fdata()

# Read the TSV file into a DataFrame
atlas_datas = pd.read_csv(tsv_file_path, delimiter='\t')
atlas_labels=atlas_datas['name'].tolist

# Initialize lists to store coordinates
x_coords, y_coords, z_coords = [], [], []

# Loop through each unique region ID in the atlas (excluding background if ID = 0)
region_ids = np.unique(atlas_data)
region_ids = region_ids[region_ids != 0]  # Remove background ID

for region_id in region_ids:
    # Create a binary mask for the region
    region_mask = atlas_data == region_id
    
    # Calculate the center of mass (centroid) for the region
    com = center_of_mass(region_mask)
    
    # Append coordinates to lists
    x_coords.append(com[0])
    y_coords.append(com[1])
    z_coords.append(com[2])

# Convert to numpy arrays
x_coords = np.array(x_coords)
y_coords = np.array(y_coords)
z_coords = np.array(z_coords)




In [ ]:
import numpy as np
from scipy.stats import pearsonr


Dice_value = min_T1   # Your list of 100 numerical values

# Calculate correlations
corr_x, p_x = pearsonr(Dice_value, x_coords)
corr_y, p_y = pearsonr(Dice_value, y_coords)
corr_z, p_z = pearsonr(Dice_value, z_coords)

# Create subplots for each coordinate
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Scatter plot for X-coordinate vs Dice values
axs[0].scatter(x_coords, Dice_value, color='b', alpha=0.6)
axs[0].set_xlabel("X Coordinate")
axs[0].set_ylabel("Dice Values of T1")
axs[0].set_title(f"X Coordinate vs Dice Values\nPearson Correlation: {corr_x:.2f} (p-value: {p_x:.3f})")

# Scatter plot for Y-coordinate vs Dice values
axs[1].scatter(y_coords, Dice_value, color='g', alpha=0.6)
axs[1].set_xlabel("Y Coordinate")
axs[1].set_ylabel("Dice Values of T1")
axs[1].set_title(f"Y Coordinate vs Dice Values\nPearson Correlation: {corr_y:.2f} (p-value: {p_y:.3f})")

# Scatter plot for Z-coordinate vs Dice values
axs[2].scatter(z_coords, Dice_value, color='r', alpha=0.6)
axs[2].set_xlabel("Z Coordinate")
axs[2].set_ylabel("Dice Values of T1")
axs[2].set_title(f"Z Coordinate vs Dice Values\nPearson Correlation: {corr_z:.2f} (p-value: {p_z:.3f})")

# Adjust layout and show plot
plt.tight_layout()
plt.show()



In [ ]:
Dice_value=min_T1

# Define the reference point (e.g., origin or center of brain)
reference_point = np.array([0, 0, 0])

# Calculate Euclidean distances from each region's (x, y, z) to the reference point
distances = np.sqrt((x_coords - reference_point[0])**2 + 
                    (y_coords - reference_point[1])**2 + 
                    (z_coords - reference_point[2])**2)

# Calculate the correlation between Dice values and distances
corr_dist, p_dist = pearsonr(Dice_value, distances)
print(f"Correlation between Dice values and Euclidean distance: {corr_dist:.2f} (p-value: {p_dist:.3f})")

# Plot Dice values vs. Euclidean distance
plt.figure(figsize=(8, 6))
plt.scatter(distances, Dice_value, color='purple', alpha=0.6)
plt.xlabel("Euclidean Distance from Reference Point")
plt.ylabel("Dice Values of T1")
plt.title(f"Dice Values vs Euclidean Distance\nPearson Correlation: {corr_dist:.2f} (p-value: {p_dist:.3f})")
plt.show()
